In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os
import os.path
base_dir = '../data_repro'
tgt_dir = '../figures_repro'
os.makedirs(tgt_dir, exist_ok=True)

# Determination of OPEP Models

In [3]:
from __future__ import print_function

%config InlineBackend.figure_format = 'retina'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pylab
import seaborn as sns
import math
import statistics
from scipy.stats import sem
from scipy import stats

from pathlib import Path

import statsmodels.api as sm

import sklearn
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from sklearn.utils import check_consistent_length
from sklearn.utils import check_array
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_validate

from sklearn.model_selection import cross_val_score
from sklearn.base import BaseEstimator, RegressorMixin

plt.style.use("seaborn-ticks") 
sns.set_context(context = "paper", font_scale = 2)

/tmp/ipykernel_2330055/2428978188.py:31: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn-ticks")


Reading in the CSVs with experimental acceptor-donor pairs and DFT/sTD-DFT calculated properties with the original dataset

In [4]:
df = pd.read_csv(os.path.join(base_dir, 'DFT_for_analysis.csv'))
print("Number of acceptor-donor pairs is " + str(len(df.index)))

df_highPCE = pd.read_csv(os.path.join(base_dir, 'DFT_for_analysis_highPCE.csv'))
print("Number of acceptor-donor pairs with PCE greater than 9% is " + str(len(df_highPCE.index)))

Number of acceptor-donor pairs is 85
Number of acceptor-donor pairs with PCE greater than 9% is 60


Setting each descriptor to floats and converting into a DataFrame

In [5]:
DFTfields = {k: float for k in [
    'AccHOMO', 'AccCalcLUMO', 'AccFundBg', 'AccOptBg', 'AccEnergyTransitioneV', 'AccTriplet', 
    'AccDeltaHOMO', 'AccDeltaLUMO', 'DeltaDipMom', 'GSDipMom', 'AccEnergyTransitionWavenumber', 'AccOscStr', 
    'AccSumOscStr', 'AccElectroIndex', 'AccNucleoIndex', 'AccChemHard', 
    'AccElectrodonating', 'AccElectroaccepting', 'PiSystemSize', 'Planarity', 'Polarizability',
    'DonHOMO', 'DonCalcLUMO', 'DonFundBg', 'DonEnergyTransitioneV', 'DonDeltaHOMO', 'DonDeltaLUMO',
    'DonEnergyTransitionWavenumber', 'DonOscStr', 'DonSumOscStr', 'DonElectroIndex', 'DonNucleoIndex', 'DonChemHard',
    'DonElectrodonating', 'DonElectroaccepting',
    'AbsFOM',    'LUMOOffset', 'HOMOOffset', 'DHomoALumoOffset', 
    'acc_first_oscs', 'acc_highest_oscs_under_ten', 'acc_lowest_transition_eV',
    'acc_lowest_transition_wavenumber', 'don_first_oscs', 'don_highest_oscs_under_ten',
    'don_lowest_transition_eV', 'don_lowest_transition_wavenumber',
    'sTDDFTLUMO', 'sTDDFTHOMO', 'sTDDFTdeltaLUMO', 'sTDDFTdeltaHOMO', 'sTDDFToptbg', 
    'sTDDFToscs', 'sTDDFTfundbg', 'sTDDFTsinglepointenergy', 'sTDDFTdipolemoment', 'sTDDFTsummedoscs', 
    'sTDDFTabsFOM', 'sTDDFTAccNucleoIndex', 'sTDDFTaccElectrophilicity', 'sTDDFTAccChemHard', 
    'sTDDFTAccElectrodonating', 'sTDDFTAccElectroaccepting',
    'donsTDDFTLUMO', 'donsTDDFTHOMO', 'sTDDFTDonNucleoIndex', 'donsTDDFTdeltaLUMO', 'donsTDDFTdeltaHOMO', 
    'donsTDDFToptbg', 'donsTDDFToscs', 'donsTDDFTfundbg', 'donsTDDFTsinglepointenergy', 'donsTDDFTdipolemoment',
    'donsTDDFTsummedoscs', 'donsTDDFTabsFOM', 'sTDDFTDonChemHard', 
    'sTDDFTDonElectrodonating', 'sTDDFTDonElectroaccepting', 'sTDDFTdonElectrophilicity',
    'sTDDFTDHomoALumoOffset', 'sTDDFTHOMOoffset', 'sTDDFTLUMOoffset',
    'sTDDFTdonlowestengtranseV', 'sTDDFTdonlowestengtranswavenumber', 'sTDDFTdonfirstengtranseV',
    'sTDDFTdonfirstengtranswavenumber', 'sTDDFTdonfirstoscs', 'sTDDFTacclowestengtranseV',
    'sTDDFTacclowestengtranswavenumber', 'sTDDFTaccfirstengtranseV', 'sTDDFTaccfirstengtranswavenumber',
    'sTDDFTaccfirstoscs', 'sTDDFTacchighestoscsunderten', 'sTDDFTdonhighestoscsunderten',
    'ScharberJsclowestoptbg', 'ScharberVoc', 'ScharberJscDon', 'ScharberJscAcc', 'ScharberTotalJsc',
    'ScharberFF', 'ScharberPCEtotal', 'ScharberPCEacc', 'ScharberPCEdon', 'ScharberPCElowestoptbg', 
    'sTDDFTScharberVoc', 'sTDDFTScharberJscDon', 'sTDDFTScharberJscAcc', 'sTDDFTScharberJscTot', 
    'sTDDFTScharberPCETot', 'sTDDFTScharberPCEAcc', 'sTDDFTScharberPCEDon',
    'ImamuraVoc', 'ImamuraJscDon', 'ImamuraJscAcc', 'ImamuraTotalJsc', 'ImamuraFF', 'ImamuraPCEtotal', 
    'ImamuraPCEacc', 'ImamuraPCEdon', 
    'AlharbiVocDon', 'AlharbiVocAcc', 'AlharbiFFDon', 'AlharbiFFAcc', 'AlharbiPCEAcc', 'AlharbiPCEDon',
    'ExperimentalFF', 'ExperimentalVoc', 'ExperimentalJsc', 'ExperimentalPCE']
}
df = df.astype(DFTfields)
df_highPCE = df_highPCE.astype(DFTfields)

In [6]:

## from sci-kit-learn https://github.com/scikit-learn/scikit-learn/blob/4773f3e39/sklearn/metrics/_regression.py#L197
def _check_reg_targets(y_true, y_pred, multioutput, dtype="numeric"):
    check_consistent_length(y_true, y_pred)
    y_true = check_array(y_true, ensure_2d=False, dtype=dtype)
    y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)

    if y_true.ndim == 1:
        y_true = y_true.reshape((-1, 1))

    if y_pred.ndim == 1:
        y_pred = y_pred.reshape((-1, 1))

    if y_true.shape[1] != y_pred.shape[1]:
        raise ValueError("y_true and y_pred have different number of output "
                         "({0}!={1})".format(y_true.shape[1], y_pred.shape[1]))

    n_outputs = y_true.shape[1]
    allowed_multioutput_str = ('raw_values', 'uniform_average',
                               'variance_weighted')
    if isinstance(multioutput, str):
        if multioutput not in allowed_multioutput_str:
            raise ValueError("Allowed 'multioutput' string values are {}. "
                             "You provided multioutput={!r}".format(
                                 allowed_multioutput_str,
                                 multioutput))
    elif multioutput is not None:
        multioutput = check_array(multioutput, ensure_2d=False)
        if n_outputs == 1:
            raise ValueError("Custom weights are useful only in "
                             "multi-output cases.")
        elif n_outputs != len(multioutput):
            raise ValueError(("There must be equally many custom weights "
                              "(%d) as outputs (%d).") %
                             (len(multioutput), n_outputs))
    y_type = 'continuous' if n_outputs == 1 else 'continuous-multioutput'

    return y_type, y_true, y_pred, multioutput

def mean_absolute_percentage_error(y_true, y_pred,
                                   sample_weight=None,
                                   multioutput='uniform_average'):

    y_type, y_true, y_pred, multioutput = _check_reg_targets(
        y_true, y_pred, multioutput)
    check_consistent_length(y_true, y_pred, sample_weight)
    epsilon = np.finfo(np.float64).eps
    mape = np.abs(y_pred - y_true) / np.maximum(np.abs(y_true), epsilon)
    output_errors = np.average(mape,
                               weights=sample_weight, axis=0)
    if isinstance(multioutput, str):
        if multioutput == 'raw_values':
            return output_errors
        elif multioutput == 'uniform_average':
            # pass None as weights to np.average: uniform mean
            multioutput = None

    return np.average(output_errors, weights=multioutput)

def evaluate_model(list_of_desc, target, dataframe, cf = False, output=True):
    '''
    Evaluates the errors of the proposed model
    
    Parameters
    ---------
    list_of_desc = name of the list that contains the descriptors
    target = target experimental value in the dataframe. Ex: df_highPCE['ExperimentalJsc']
    dataframe = name of dataframe to use descriptors from
    cf = should it show the coefficients
    
    Returns
    -------
    R2, MAE, RMSE, MAPE, y-intercept of equation, coefficients of each descriptor, average predictions
    '''
    
    new_df = dataframe[list_of_desc].copy()
    
    max_mae = 10.0
    best_alpha = 0
    for a in range(1,200):
        lm =Lasso(alpha=float(a)/100.0)
        lm.fit(new_df, target)
        mae = mean_absolute_error(target, lm.predict(new_df))
        if mae < max_mae:
            best_alpha = float(a)/100.0
            max_mae = mae

    r2_score = []
    intercepts = []
    coefs = []
    predicted_PCE = []
    MAE_cv = []
    RMSE_cv = []

    PCE_cv = cross_validate(Lasso(alpha = best_alpha, max_iter = 100000), new_df, target, cv=5, scoring = ['neg_mean_squared_error'] , return_estimator = True)
    for x in PCE_cv['estimator']:
        coef = x.coef_
        coefs.append(coef)

        pred = x.predict(new_df)
        predicted_PCE.append(pred)

        r2_score.append(x.score(new_df, target))
        intercepts.append(x.intercept_)

        
    avg_coef = sum(coefs)/5.0    
    zipped_coefs = zip(list_of_desc, avg_coef)
    
    if output == True:
        if cf == True:
            print('The average intercept from 5 folds is ', statistics.mean(intercepts))
            print('The average coefficients of the 5 folds is ', list(zipped_coefs))
    
    coefficients = list(zipped_coefs)
    
    avg_pred = sum(predicted_PCE)/ 5.0    
    all_mae = []
    all_RMSE = []
    all_MAPE = []
    for x in range(len(predicted_PCE)):
        mae = mean_absolute_error(target, predicted_PCE[x])
        all_mae.append(mae)
        mse = mean_squared_error(target, predicted_PCE[x])
        rmse = math.sqrt(mse)
        all_RMSE.append(rmse)
        mape = mean_absolute_percentage_error(target, predicted_PCE[x]) * 100
        all_MAPE.append(mape)
    SEM_mae = round(sem(all_mae), 3)
    SEM_RMSE = round(sem(all_RMSE), 5)
    SEM_MAPE = round(sem(all_MAPE), 5)
    
    
    mean_MAE = round(statistics.mean(all_mae), 3)
    mean_RMSE = round(statistics.mean(all_RMSE), 4)
    mean_MAPE = round(statistics.mean(all_MAPE), 4)
    SEM_r2 = round(sem(r2_score), 3)
    
    if output == True:
        print('The 5-fold cross-validated RMSE of this model is, ', mean_RMSE, ' +/- ', SEM_RMSE)
        print('The 5-fold cross-validated MAE of the average ', mean_MAE, ' +/- ', SEM_mae)
        print('The average r^2 value is ', statistics.mean(r2_score), ' +/- ', SEM_r2 )
    mean_r2 = round(statistics.mean(r2_score), 3)
    
    intercept = statistics.mean(intercepts)
    
    return mean_r2, SEM_r2, mean_MAE, SEM_mae, mean_RMSE, SEM_RMSE, mean_MAPE, SEM_MAPE, intercept, coefficients, avg_pred

In [7]:
# Will tune the descriptors down to 6 until to achieve an optimized RMSE
# returns a list of the top 6 descriptors
def optimize_eq(method, df, target):
    
    if method == "DFT":
        # all descriptors examined for DFT/TD-DFT
        all_descriptors = [
            'AccHOMO','AccCalcLUMO','AccFundBg','AccOptBg','AccEnergyTransitioneV','AccTriplet', 'AccDeltaHOMO','AccDeltaLUMO','DeltaDipMom','GSDipMom',
            'AccEnergyTransitionWavenumber', 'AccOscStr', 'AccSumOscStr','AccElectroIndex','AccNucleoIndex', 'AccChemHard', 'AccElectrodonating',
            'AccElectroaccepting','PiSystemSize', 'Planarity', 'Polarizability','DonHOMO', 'DonCalcLUMO', 'DonFundBg', 'DonEnergyTransitioneV',
            'DonDeltaHOMO', 'DonDeltaLUMO','DonEnergyTransitionWavenumber','DonOscStr', 'DonSumOscStr','DonElectroIndex', 'DonNucleoIndex','DonChemHard',
            'DonElectrodonating', 'DonElectroaccepting','AbsFOM','LUMOOffset', 'HOMOOffset', 'DHomoALumoOffset','acc_first_oscs','acc_highest_oscs_under_ten',
            'acc_lowest_transition_eV','acc_lowest_transition_wavenumber','don_first_oscs','don_highest_oscs_under_ten','don_lowest_transition_eV',
            'don_lowest_transition_wavenumber']
    elif method == 'sTD-DFT':
        # all descriptors examined for sTD-DFT
        all_descriptors = [
            'sTDDFTLUMO','sTDDFTHOMO','sTDDFTdeltaLUMO','sTDDFTdeltaHOMO','sTDDFToptbg','sTDDFToscs','sTDDFTfundbg','sTDDFTsinglepointenergy',
            'sTDDFTdipolemoment','donsTDDFTLUMO','donsTDDFTHOMO','donsTDDFTdeltaLUMO','donsTDDFTdeltaHOMO','donsTDDFToptbg','donsTDDFToscs','donsTDDFTfundbg',
            'donsTDDFTsinglepointenergy','donsTDDFTdipolemoment','sTDDFTDHomoALumoOffset','sTDDFTaccElectrophilicity','sTDDFTdonElectrophilicity',
            'sTDDFTHOMOoffset','sTDDFTLUMOoffset','Planarity','PiSystemSize','sTDDFTsummedoscs','sTDDFTabsFOM','donsTDDFTsummedoscs','donsTDDFTabsFOM',
            'sTDDFTAccChemHard','sTDDFTAccElectrodonating','sTDDFTAccElectroaccepting','sTDDFTDonChemHard','sTDDFTDonElectrodonating','sTDDFTDonElectroaccepting',
            'sTDDFTdonlowestengtranseV', 'sTDDFTdonlowestengtranswavenumber', 'sTDDFTdonfirstengtranseV', 'sTDDFTdonfirstengtranswavenumber', 'sTDDFTdonfirstoscs',
            'sTDDFTacclowestengtranseV', 'sTDDFTacclowestengtranswavenumber', 'sTDDFTaccfirstengtranseV', 'sTDDFTaccfirstengtranswavenumber', 'sTDDFTaccfirstoscs',
            'sTDDFTacchighestoscsunderten', 'sTDDFTdonhighestoscsunderten'] 
    else:
        return 'Not a valid method. Options are DFT or sTD-DFT'
    
    # LASSO to remove any with coefficients of zero
    descriptors_to_remove = []
    results = evaluate_model(all_descriptors, df[target], df,output=False)
    for x in range(len(results[9])):
        if results[9][x][1] == 0.0:
            descriptors_to_remove.append(results[9][x][0])
            
    for x in descriptors_to_remove:
        all_descriptors.remove(x)
    print(all_descriptors)
    
    while len(all_descriptors) > 6:
        rmse = 100.0
        diff = 0.0
        mae = 100.0
        r2 = -1.00
        for x in range(len(all_descriptors)):
            temp_descriptor = all_descriptors[x]
            new_list = all_descriptors.copy()
            new_list.remove(temp_descriptor)
            results = evaluate_model(new_list, df[target], df,output=False)
            new_diff = 100.0 - float(results[4])
            
            if new_diff >= diff:
                if new_diff == diff:
                    if mae == results[2]:
                        if results[0] > r2:
                            diff = new_diff
                            mae = results[2]
                            r2 = results[0]
                            rmse = results[4]
                            best_list = new_list
                    elif mae > results[2]:
                        diff = new_diff
                        mae = results[2]
                        r2 = results[0]
                        rmse = results[4]
                        best_list = new_list
                else:
                    diff = new_diff
                    mae = results[2]
                    r2 = results[0]
                    rmse = results[4]
                    best_list = new_list
                
        all_descriptors = best_list
        print(len(all_descriptors))
        print(rmse)
        
    print(all_descriptors)
    print(rmse)
    
    return all_descriptors

This is a list of all of the DFT descriptors

In [8]:
descriptors = [
    'AccHOMO', 'AccCalcLUMO', 'AccFundBg', 'AccOptBg', 'AccEnergyTransitioneV', 'AccTriplet', 'AccDeltaHOMO', 'AccDeltaLUMO',
    'DeltaDipMom', 'GSDipMom', 'AccEnergyTransitionWavenumber', 'AccOscStr', 'AccSumOscStr', 'AccElectroIndex', 'AccNucleoIndex',
    'AccChemHard', 'AccElectrodonating', 'AccElectroaccepting', 'PiSystemSize', 'Planarity', 'Polarizability', 'DonHOMO',
    'DonCalcLUMO', 'DonFundBg', 'DonEnergyTransitioneV', 'DonDeltaHOMO', 'DonDeltaLUMO', 'DonEnergyTransitionWavenumber',
    'DonOscStr', 'DonSumOscStr', 'DonElectroIndex', 'DonNucleoIndex', 'DonChemHard', 'DonElectrodonating', 'DonElectroaccepting',
    'AbsFOM', 'LUMOOffset', 'HOMOOffset', 'DHomoALumoOffset']
print(len(descriptors))

39


# Analysis of Descriptors for OPEP Models

Each model has two methods to select descriptors. You can choose which descriptors to add, by adding it to the "best_descriptors" list, or we can use code to choose the best descriptors (optimize_eq() )

## OPEP/B3LYP Jsc for PCE >9%

In [9]:
# best_descriptors = optimize_eq('DFT', df_highPCE, 'ExperimentalJsc')
best_descriptors = ['DeltaDipMom', 'AccElectrodonating', 'DonEnergyTransitionWavenumber', 'DonElectrodonating', 'acc_highest_oscs_under_ten', 'acc_lowest_transition_wavenumber']
# + AccElectrodonating
# + DonEnergyTransitionWavenumber
# + acc_highest_oscs_under_ten
# - Polarizability
# - acc_first_oscs
# - don_lowest_transition_wavenumber
LassoJsc_highPCE = evaluate_model(best_descriptors, df_highPCE['ExperimentalJsc'], df_highPCE, cf=True)

The average intercept from 5 folds is  -104.17582463431191
The average coefficients of the 5 folds is  [('DeltaDipMom', 2.008010632877519), ('AccElectrodonating', 0.8174316025834877), ('DonEnergyTransitionWavenumber', 0.003143723257703116), ('DonElectrodonating', 6.198174299860243), ('acc_highest_oscs_under_ten', 1.6603789880117354), ('acc_lowest_transition_wavenumber', -0.0006223719401219061)]
The 5-fold cross-validated RMSE of this model is,  2.0187  +/-  0.01247
The 5-fold cross-validated MAE of the average  1.61  +/-  0.016
The average r^2 value is  0.6636633650759629  +/-  0.004


## OPEP/B3LYP Jsc for any PCE

In [10]:
#best_descriptors = optimize_eq('DFT', df, 'ExperimentalJsc')
best_descriptors = ['AccTriplet', 'DeltaDipMom', 'AccElectrodonating', 'DonSumOscStr', 'acc_first_oscs', 'don_lowest_transition_wavenumber']
# + DonSumOscStr
# + don_lowest_transition_wavenumber
# - DonElectrodonating
# - acc_lowest_transition_wavenumber
LassoJsc = evaluate_model(best_descriptors, df['ExperimentalJsc'], df, cf=True)

The average intercept from 5 folds is  -23.35167514719245
The average coefficients of the 5 folds is  [('AccTriplet', 14.098584322361049), ('DeltaDipMom', 2.427201240636369), ('AccElectrodonating', 2.1590413713563352), ('DonSumOscStr', -0.6239669335336389), ('acc_first_oscs', 3.3890988944248406), ('don_lowest_transition_wavenumber', -0.0008523191155176775)]
The 5-fold cross-validated RMSE of this model is,  3.8404  +/-  0.02102
The 5-fold cross-validated MAE of the average  2.85  +/-  0.016
The average r^2 value is  0.5312872913307443  +/-  0.005


## OPEP/B3LYP FF for PCE >9%

In [11]:
#best_descriptors = optimize_eq('DFT', df_highPCE, 'ExperimentalFF')
best_descriptors = ['AccTriplet', 'AccElectrodonating', 'Polarizability', 'DonElectrodonating', 'acc_lowest_transition_wavenumber', 'don_lowest_transition_wavenumber']
# + AccElectrodonating
# - AccElectroaccepting
LassoFF_highPCE = evaluate_model(best_descriptors, df_highPCE['ExperimentalFF'], df_highPCE, cf=True)

The average intercept from 5 folds is  -170.94367214256104
The average coefficients of the 5 folds is  [('AccTriplet', 30.24725324112694), ('AccElectrodonating', 1.302223294267889), ('Polarizability', -0.007922795530606503), ('DonElectrodonating', 11.895023870376479), ('acc_lowest_transition_wavenumber', -0.0016079012283429343), ('don_lowest_transition_wavenumber', 0.005496190164245867)]
The 5-fold cross-validated RMSE of this model is,  4.1773  +/-  0.02616
The 5-fold cross-validated MAE of the average  3.195  +/-  0.033
The average r^2 value is  0.40105441765676614  +/-  0.007


## OPEP/B3LYP FF for any PCE

In [12]:
# best_descriptors = optimize_eq('DFT', df, 'ExperimentalFF')
best_descriptors = ['AccTriplet', 'AccElectrodonating', 'Polarizability', 'AbsFOM', 'acc_first_oscs', 'acc_lowest_transition_wavenumber']
# + AccElectrodonating
# - AccElectroaccepting
LassoFF = evaluate_model(best_descriptors, df['ExperimentalFF'], df, cf=True)

The average intercept from 5 folds is  -22.21609320234735
The average coefficients of the 5 folds is  [('AccTriplet', 54.308775488360105), ('AccElectrodonating', 3.040622080770519), ('Polarizability', -0.007797746981169249), ('AbsFOM', 0.08356399033583692), ('acc_first_oscs', 2.5550638205001386), ('acc_lowest_transition_wavenumber', -0.0019363817449869068)]
The 5-fold cross-validated RMSE of this model is,  5.7005  +/-  0.02069
The 5-fold cross-validated MAE of the average  4.388  +/-  0.049
The average r^2 value is  0.3073519471938869  +/-  0.005


## OPEP/B3LYP Voc for PCE >9%

In [13]:
# best_descriptors = optimize_eq('DFT', df_highPCE, 'ExperimentalVoc')
best_descriptors = ['GSDipMom', 'AccOscStr', 'AccElectroaccepting', 'Polarizability', 'AbsFOM', 'acc_lowest_transition_wavenumber']
# + AccElectroaccepting
# + AbsFOM
# - AccElectroIndex
# - acc_first_oscs
LassoVoc_highPCE = evaluate_model(best_descriptors, df_highPCE['ExperimentalVoc'], df_highPCE, cf=True)

The average intercept from 5 folds is  0.8164785065227951
The average coefficients of the 5 folds is  [('GSDipMom', -0.005601094415580152), ('AccOscStr', -0.021465541235126652), ('AccElectroaccepting', -0.03454425198760526), ('Polarizability', 0.0001603707858245352), ('AbsFOM', 0.0001734267674738398), ('acc_lowest_transition_wavenumber', 2.227446565615461e-05)]
The 5-fold cross-validated RMSE of this model is,  0.0622  +/-  0.00043
The 5-fold cross-validated MAE of the average  0.05  +/-  0.001
The average r^2 value is  0.5590917575105017  +/-  0.006


## OPEP/B3LYP Voc for any PCE

In [14]:
#best_descriptors = optimize_eq('DFT', df, 'ExperimentalVoc')
best_descriptors = ['AccOscStr', 'AccElectrodonating', 'Polarizability', 'DonEnergyTransitionWavenumber', 'acc_lowest_transition_wavenumber', 'don_lowest_transition_wavenumber']
# + AccOscStr
# + DonEnergyTransitionWavenumber
# - GSDipMom
# - AbsFOM
LassoVoc = evaluate_model(best_descriptors, df['ExperimentalVoc'], df, cf=True)

The average intercept from 5 folds is  1.4648766121269727
The average coefficients of the 5 folds is  [('AccOscStr', -0.004602497538089413), ('AccElectrodonating', -0.045982143340870404), ('Polarizability', 0.00011337783219678172), ('DonEnergyTransitionWavenumber', -1.4035889973385918e-05), ('acc_lowest_transition_wavenumber', 1.7724928040162616e-05), ('don_lowest_transition_wavenumber', -1.329340643388882e-06)]
The 5-fold cross-validated RMSE of this model is,  0.09  +/-  0.00212
The 5-fold cross-validated MAE of the average  0.068  +/-  0.001
The average r^2 value is  0.36034143654616674  +/-  0.031


## OPEP/B3LYP PCE for PCE >9%

In [15]:
#best_descriptors = optimize_eq('DFT', df_highPCE, 'ExperimentalPCE')
best_descriptors = ['AccTriplet', 'DeltaDipMom', 'AccEnergyTransitionWavenumber', 'Polarizability', 'DonElectrodonating', 'don_lowest_transition_wavenumber']
# Same as before.
LassoPCE_highPCE = evaluate_model(best_descriptors, df_highPCE['ExperimentalPCE'], df_highPCE, cf=True)

The average intercept from 5 folds is  -132.49824072936747
The average coefficients of the 5 folds is  [('AccTriplet', 6.188165933906566), ('DeltaDipMom', 0.9214191124375894), ('AccEnergyTransitionWavenumber', -0.00046184213102377317), ('Polarizability', -0.003728378569027087), ('DonElectrodonating', 8.065102545730037), ('don_lowest_transition_wavenumber', 0.0038187990019674448)]
The 5-fold cross-validated RMSE of this model is,  1.5431  +/-  0.02969
The 5-fold cross-validated MAE of the average  1.246  +/-  0.014
The average r^2 value is  0.6337557106620769  +/-  0.014


## OPEP/B3LYP PCE for any PCE

In [16]:
#best_descriptors = optimize_eq('DFT', df, 'ExperimentalPCE')
best_descriptors = ['AccTriplet', 'AccEnergyTransitionWavenumber', 'AccElectroaccepting', 'Planarity', 'DonElectrodonating', 'don_lowest_transition_wavenumber']
# + AccEnergyTransitionWavenumber
# + AccElectroaccepting
# + Planarity
# - DeltaDipMom
# - AccOscStr
# - acc_lowest_transition_wavenumber
LassoPCE = evaluate_model(best_descriptors, df['ExperimentalPCE'], df, cf=True)

The average intercept from 5 folds is  -113.37551553332283
The average coefficients of the 5 folds is  [('AccTriplet', 28.716853177317425), ('AccEnergyTransitionWavenumber', -0.0010893025482155436), ('AccElectroaccepting', 1.6372883218692345), ('Planarity', -1.7225065581668104), ('DonElectrodonating', 5.399151605641703), ('don_lowest_transition_wavenumber', 0.002173614229430385)]
The 5-fold cross-validated RMSE of this model is,  2.5562  +/-  0.01066
The 5-fold cross-validated MAE of the average  1.951  +/-  0.016
The average r^2 value is  0.5475034082086669  +/-  0.004


# OPEP/sTD-DFT models

Two different datasets were used. The first is on the same acceptor-donor pairs as the OPEP/B3LYP dataset. The second has additional donors so there is a larger dataset, referred to as the "Expanded" models

In [17]:
df_sTD = pd.read_csv(os.path.join(base_dir, 'sTDDFT_for_analysis.csv'))  #includes additional acceptor-donor pairs not in the original DFT data
print("Number of acceptor-donor pairs is " + str(len(df_sTD.index)))

df_sTD_highPCE = pd.read_csv(os.path.join(base_dir, 'sTDDFT_for_analysis_highPCE.csv')) #includes additional acceptor-donor pairs not in the original DFT data, but all have PCE > 9%
print("Number of acceptor-donor pairs with experimental PCE above 9% is " + str(len(df_sTD_highPCE.index)))

Number of acceptor-donor pairs is 107
Number of acceptor-donor pairs with experimental PCE above 9% is 77


In [18]:
sTDDFT_fields = {k: float for k in [
    'sTDDFTLUMO', 'sTDDFTHOMO', 'sTDDFTdeltaLUMO', 'sTDDFTdeltaHOMO', 'sTDDFToptbg', 
    'sTDDFToscs', 'sTDDFTfundbg', 'sTDDFTsinglepointenergy', 'sTDDFTdipolemoment', 'sTDDFTsummedoscs', 'sTDDFTabsFOM',
    'sTDDFTScharberVoc', 'sTDDFTScharberJscDon', 'sTDDFTScharberJscAcc', 'sTDDFTScharberJscTot', 'sTDDFTScharberPCETot',
    'sTDDFTScharberPCEAcc', 'sTDDFTScharberPCEDon', 'donsTDDFTLUMO', 'donsTDDFTHOMO', 
    'donsTDDFTdeltaLUMO', 'donsTDDFTdeltaHOMO', 'donsTDDFToptbg', 'donsTDDFToscs',
    'donsTDDFTfundbg', 'donsTDDFTsinglepointenergy', 'donsTDDFTdipolemoment', 'donsTDDFTsummedoscs', 'donsTDDFTabsFOM', 
    'sTDDFTDHomoALumoOffset', 'sTDDFTaccElectrophilicity', 'sTDDFTAccChemHard',  'sTDDFTAccElectrodonating', 'sTDDFTAccElectroaccepting',
    'sTDDFTDonChemHard',  'sTDDFTDonElectrodonating', 'sTDDFTDonElectroaccepting', 
    'sTDDFTdonlowestengtranseV', 'sTDDFTdonlowestengtranswavenumber', 'sTDDFTdonfirstengtranseV', 'sTDDFTdonfirstengtranswavenumber', 'sTDDFTdonfirstoscs',
    'sTDDFTacclowestengtranseV', 'sTDDFTacclowestengtranswavenumber', 'sTDDFTaccfirstengtranseV', 'sTDDFTaccfirstengtranswavenumber', 'sTDDFTaccfirstoscs',
    'sTDDFTacchighestoscsunderten', 'sTDDFTdonhighestoscsunderten',
    'sTDDFTdonElectrophilicity', 'sTDDFTHOMOoffset', 'sTDDFTLUMOoffset', 'Planarity', 'PiSystemSize', 
    'ScharberFF', 'ExperimentalFF', 'ExperimentalVoc', 'ExperimentalJsc', 'ExperimentalPCE']
}
df_sTD = df_sTD.astype(sTDDFT_fields)
df_sTD_highPCE = df_sTD_highPCE.astype(sTDDFT_fields)

## OPEP/sTD-DFT PCE Expanded for PCE >9%

In [19]:
#best_descriptors = optimize_eq('sTD-DFT', df_sTD_highPCE, 'ExperimentalPCE')
best_descriptors = ['sTDDFTdeltaHOMO', 'sTDDFTsummedoscs', 'donsTDDFTsummedoscs', 'donsTDDFTabsFOM', 'sTDDFTDonElectrodonating', 'sTDDFTdonfirstengtranswavenumber']
# Same as before
LassoPCE_sTD_highPCE_all = evaluate_model(best_descriptors, df_sTD_highPCE['ExperimentalPCE'], df_sTD_highPCE, cf=True)

The average intercept from 5 folds is  -33.52212127896147
The average coefficients of the 5 folds is  [('sTDDFTdeltaHOMO', 4.5675886882886), ('sTDDFTsummedoscs', 1.3894415725926328), ('donsTDDFTsummedoscs', -0.4834248258906942), ('donsTDDFTabsFOM', 0.17756837345506807), ('sTDDFTDonElectrodonating', 2.5034155294359857), ('sTDDFTdonfirstengtranswavenumber', 0.0009458944945970159)]
The 5-fold cross-validated RMSE of this model is,  1.5807  +/-  0.02367
The 5-fold cross-validated MAE of the average  1.28  +/-  0.018
The average r^2 value is  0.5632539808575581  +/-  0.013


## OPEP/sTD-DFT PCE for PCE>9%

In [20]:
#best_descriptors = optimize_eq('sTD-DFT', df_highPCE, 'ExperimentalPCE')
best_descriptors = ['sTDDFTdeltaHOMO', 'donsTDDFTdipolemoment', 'sTDDFTsummedoscs', 'donsTDDFTabsFOM', 'sTDDFTdonlowestengtranswavenumber', 'sTDDFTacchighestoscsunderten']
# + sTDDFTacchighestoscsunderten
# - sTDDFTdeltaHOMO
LassoPCE_sTD_highPCE = evaluate_model(best_descriptors, df_highPCE['ExperimentalPCE'], df_highPCE, cf=True)

The average intercept from 5 folds is  99.18565697353252
The average coefficients of the 5 folds is  [('sTDDFTdeltaHOMO', 4.619186640981389), ('donsTDDFTdipolemoment', -1.6215980287842044), ('sTDDFTsummedoscs', 1.9610168729269013), ('donsTDDFTabsFOM', 0.10204002301376705), ('sTDDFTdonlowestengtranswavenumber', -0.004787821280152764), ('sTDDFTacchighestoscsunderten', -0.9939417302739179)]
The 5-fold cross-validated RMSE of this model is,  1.6773  +/-  0.0377
The 5-fold cross-validated MAE of the average  1.278  +/-  0.015
The average r^2 value is  0.5670293261428775  +/-  0.019


## OPEP/sTD-DFT PCE Expanded for any experimental PCE

In [22]:
#best_descriptors = optimize_eq('sTD-DFT', df_sTD, 'ExperimentalPCE')
best_descriptors = ['sTDDFTdeltaLUMO', 'Planarity', 'sTDDFTabsFOM', 'donsTDDFTsummedoscs', 'sTDDFTAccElectrodonating', 'sTDDFTaccfirstengtranswavenumber']
# + sTDDFTabsFOM
# + sTDDFTAccElectrodonating
# + sTDDFTaccfirstengtranswavenumber
# - sTDDFTsinglepointenergy
# - donsTDDFTHOMO
# - donsTDDFTsummedoscs
LassoPCE_sTD_all = evaluate_model(best_descriptors, df_sTD['ExperimentalPCE'], df_sTD, cf=True)

The average intercept from 5 folds is  -46.90817315885534
The average coefficients of the 5 folds is  [('sTDDFTdeltaLUMO', -7.2322904571542015), ('Planarity', -0.985205838229458), ('sTDDFTabsFOM', 0.317623296657194), ('donsTDDFTsummedoscs', 0.3420633866704573), ('sTDDFTAccElectrodonating', 3.548286045331537), ('sTDDFTaccfirstengtranswavenumber', 0.0013760998039417942)]
The 5-fold cross-validated RMSE of this model is,  2.5894  +/-  0.00591
The 5-fold cross-validated MAE of the average  2.002  +/-  0.015
The average r^2 value is  0.4748488029207805  +/-  0.002


## OPEP/sTD-DFT PCE for any experimental PCE

In [23]:
#best_descriptors = optimize_eq('sTD-DFT', df, 'ExperimentalPCE')
best_descriptors = ['sTDDFTdeltaLUMO', 'sTDDFTdeltaHOMO', 'donsTDDFTsinglepointenergy', 'sTDDFTHOMOoffset', 'Planarity', 'sTDDFTabsFOM']
# All new except 'donsTDDFTsinglepointenergy', 'Planarity'
LassoPCE_sTD = evaluate_model(best_descriptors, df['ExperimentalPCE'], df, cf=True)

The average intercept from 5 folds is  -0.27925936781307215
The average coefficients of the 5 folds is  [('sTDDFTdeltaLUMO', -6.15460612146593), ('sTDDFTdeltaHOMO', 3.4410622141340363), ('donsTDDFTsinglepointenergy', -0.00046489063483821897), ('sTDDFTHOMOoffset', 2.0524460235711794), ('Planarity', -1.472052659800481), ('sTDDFTabsFOM', 0.24736870261442107)]
The 5-fold cross-validated RMSE of this model is,  2.8037  +/-  0.02614
The 5-fold cross-validated MAE of the average  2.078  +/-  0.03
The average r^2 value is  0.4554639671973111  +/-  0.01


## OPEP/sTD-DFT Voc Expanded for PCE>9%

In [25]:
#best_descriptors = optimize_eq('sTD-DFT', df_sTD_highPCE, 'ExperimentalVoc')
best_descriptors = ['sTDDFTsinglepointenergy', 'sTDDFTdipolemoment', 'donsTDDFTdipolemoment', 'Planarity', 'sTDDFTAccElectrodonating', 'sTDDFTaccfirstengtranswavenumber']
# + sTDDFTAccElectrodonating
# - PiSystemSize
LassoVoc_sTD_highPCE_all = evaluate_model(best_descriptors, df_sTD_highPCE['ExperimentalVoc'], df_sTD_highPCE, cf=True)

The average intercept from 5 folds is  0.28317970290435446
The average coefficients of the 5 folds is  [('sTDDFTsinglepointenergy', 6.1423377356833496e-06), ('sTDDFTdipolemoment', -0.0074225827717204025), ('donsTDDFTdipolemoment', -0.0034860596211277626), ('Planarity', 0.007872227963221824), ('sTDDFTAccElectrodonating', -0.016541985585541173), ('sTDDFTaccfirstengtranswavenumber', 4.599108790271718e-05)]
The 5-fold cross-validated RMSE of this model is,  0.0611  +/-  0.0019
The 5-fold cross-validated MAE of the average  0.047  +/-  0.002
The average r^2 value is  0.5216119133730507  +/-  0.03


## OPEP/sTD-DFT Voc for experimental PCE >9%

In [26]:
#best_descriptors = optimize_eq('sTD-DFT', df_highPCE, 'ExperimentalVoc')
best_descriptors = ['sTDDFTsinglepointenergy', 'sTDDFTdipolemoment', 'donsTDDFTsinglepointenergy', 'sTDDFTAccElectrodonating', 'sTDDFTdonfirstengtranswavenumber', 'sTDDFTaccfirstengtranswavenumber']
# All different, except 'donsTDDFTsinglepointenergy', 'sTDDFTAccElectrodonating'
LassoVoc_sTD_highPCE = evaluate_model(best_descriptors, df_highPCE['ExperimentalVoc'], df_highPCE, cf=True)

The average intercept from 5 folds is  -0.27870122919946205
The average coefficients of the 5 folds is  [('sTDDFTsinglepointenergy', 9.026471908379412e-06), ('sTDDFTdipolemoment', -0.004365785602950457), ('donsTDDFTsinglepointenergy', -1.0741951779725645e-05), ('sTDDFTAccElectrodonating', -0.023631115730621346), ('sTDDFTdonfirstengtranswavenumber', 1.848060179732293e-05), ('sTDDFTaccfirstengtranswavenumber', 5.3113019952176336e-05)]
The 5-fold cross-validated RMSE of this model is,  0.0593  +/-  0.00134
The 5-fold cross-validated MAE of the average  0.047  +/-  0.002
The average r^2 value is  0.5991650615770805  +/-  0.018


## OPEP/sTD-DFT Voc Expanded for any experimental PCE

In [27]:
#best_descriptors = optimize_eq('sTD-DFT', df_sTD, 'ExperimentalVoc')
best_descriptors = ['sTDDFTdipolemoment', 'donsTDDFTsinglepointenergy', 'sTDDFTsummedoscs', 'sTDDFTabsFOM', 'sTDDFTAccElectrodonating', 'sTDDFTaccfirstengtranswavenumber']
# + sTDDFTsummedoscs
# - sTDDFTdonlowestengtranswavenumber
LassoVoc_sTD_all = evaluate_model(best_descriptors, df_sTD['ExperimentalVoc'], df_sTD, cf=True)

The average intercept from 5 folds is  -0.0179301695289104
The average coefficients of the 5 folds is  [('sTDDFTdipolemoment', -0.005396857414028858), ('donsTDDFTsinglepointenergy', -1.0335890264958738e-05), ('sTDDFTsummedoscs', -0.0041431883375620095), ('sTDDFTabsFOM', 0.002471823089869942), ('sTDDFTAccElectrodonating', -0.026607785162460356), ('sTDDFTaccfirstengtranswavenumber', 5.594740346074583e-05)]
The 5-fold cross-validated RMSE of this model is,  0.0765  +/-  0.00187
The 5-fold cross-validated MAE of the average  0.058  +/-  0.002
The average r^2 value is  0.4709782564007521  +/-  0.026


## OPEP/sTD-DFT Voc for any experimental PCE

In [28]:
#best_descriptors = optimize_eq('sTD-DFT', df, 'ExperimentalVoc')
best_descriptors = ['donsTDDFTsinglepointenergy', 'sTDDFTabsFOM', 'donsTDDFTabsFOM', 'sTDDFTAccElectrodonating', 'sTDDFTdonlowestengtranswavenumber', 'sTDDFTaccfirstengtranswavenumber']
# + sTDDFTdonlowestengtranswavenumber
# + sTDDFTaccfirstengtranswavenumber'
# - donsTDDFTdipolemoment
# - PiSystemSize
LassoVoc_sTD= evaluate_model(best_descriptors, df['ExperimentalVoc'], df, cf=True)

The average intercept from 5 folds is  -0.49049065190272634
The average coefficients of the 5 folds is  [('donsTDDFTsinglepointenergy', -3.091586614153662e-05), ('sTDDFTabsFOM', 0.0035621654184991255), ('donsTDDFTabsFOM', -0.005338509935033256), ('sTDDFTAccElectrodonating', -0.0427411952232617), ('sTDDFTdonlowestengtranswavenumber', 1.790050915997397e-05), ('sTDDFTaccfirstengtranswavenumber', 6.043278888413694e-05)]
The 5-fold cross-validated RMSE of this model is,  0.0685  +/-  0.00196
The 5-fold cross-validated MAE of the average  0.051  +/-  0.001
The average r^2 value is  0.6288221952627925  +/-  0.021


## OPEP/sTD-DFT Jsc Expanded for experimental PCE >9%

In [29]:
#best_descriptors = optimize_eq('sTD-DFT', df_sTD_highPCE, 'ExperimentalJsc')
best_descriptors = ['sTDDFTdeltaLUMO', 'sTDDFTdeltaHOMO', 'donsTDDFTdeltaLUMO', 'sTDDFTaccElectrophilicity', 'sTDDFTsummedoscs', 'donsTDDFTabsFOM']
# + sTDDFTdeltaHOMO
# + sTDDFTaccElectrophilicity
# - sTDDFTDonElectrodonating
# - sTDDFTaccfirstengtranswavenumber
LassoJsc_sTD_highPCE_all = evaluate_model(best_descriptors, df_sTD_highPCE['ExperimentalJsc'], df_sTD_highPCE, cf=True)

The average intercept from 5 folds is  -26.545245884062997
The average coefficients of the 5 folds is  [('sTDDFTdeltaLUMO', -7.1084524775197835), ('sTDDFTdeltaHOMO', 5.874037323776124), ('donsTDDFTdeltaLUMO', 6.774403759451291), ('sTDDFTaccElectrophilicity', 4.748518119238025), ('sTDDFTsummedoscs', 2.3118738532439504), ('donsTDDFTabsFOM', 0.08953212184175999)]
The 5-fold cross-validated RMSE of this model is,  1.8309  +/-  0.0393
The 5-fold cross-validated MAE of the average  1.461  +/-  0.027
The average r^2 value is  0.7117652614254684  +/-  0.013


## OPEP/sTD-DFT Jsc for experimental PCE >9%

In [30]:
#best_descriptors = optimize_eq('sTD-DFT', df_highPCE, 'ExperimentalJsc')
best_descriptors = ['donsTDDFTsinglepointenergy', 'donsTDDFTdipolemoment', 'sTDDFTsummedoscs', 'donsTDDFTabsFOM', 'sTDDFTdonlowestengtranswavenumber', 'sTDDFTaccfirstengtranswavenumber']
# + donsTDDFTdipolemoment
# + sTDDFTdonlowestengtranswavenumber
# + sTDDFTaccfirstengtranswavenumber
# - sTDDFTdeltaLUMO
# - sTDDFTdeltaHOMO
# - sTDDFTaccElectrophilicity
LassoJsc_sTD_highPCE= evaluate_model(best_descriptors, df_highPCE['ExperimentalJsc'], df_highPCE, cf=True)

The average intercept from 5 folds is  169.3829043445544
The average coefficients of the 5 folds is  [('donsTDDFTsinglepointenergy', 0.001617912791518985), ('donsTDDFTdipolemoment', -1.840404782582123), ('sTDDFTsummedoscs', 3.2170502655935804), ('donsTDDFTabsFOM', 0.48253061772160893), ('sTDDFTdonlowestengtranswavenumber', -0.006748995694106414), ('sTDDFTaccfirstengtranswavenumber', -0.001068853673189572)]
The 5-fold cross-validated RMSE of this model is,  2.0051  +/-  0.02801
The 5-fold cross-validated MAE of the average  1.523  +/-  0.028
The average r^2 value is  0.6679681013397512  +/-  0.009


## OPEP/sTD-DFT Jsc Expanded for any experimental PCE

In [31]:
#best_descriptors = optimize_eq('sTD-DFT', df_sTD, 'ExperimentalJsc')
best_descriptors = ['sTDDFTdeltaLUMO', 'sTDDFTdeltaHOMO', 'sTDDFTsinglepointenergy', 'Planarity', 'donsTDDFTsummedoscs', 'sTDDFTAccElectrodonating']
# Same as before.
LassoJsc_sTD_all= evaluate_model(best_descriptors, df_sTD['ExperimentalJsc'], df_sTD, cf=True)

The average intercept from 5 folds is  -31.834406147961992
The average coefficients of the 5 folds is  [('sTDDFTdeltaLUMO', -5.47769176002192), ('sTDDFTdeltaHOMO', 5.163118053564978), ('sTDDFTsinglepointenergy', -0.0010339992438999016), ('Planarity', -2.331542983540579), ('donsTDDFTsummedoscs', 0.33294623317388083), ('sTDDFTAccElectrodonating', 5.32275974708278)]
The 5-fold cross-validated RMSE of this model is,  3.4879  +/-  0.04085
The 5-fold cross-validated MAE of the average  2.508  +/-  0.031
The average r^2 value is  0.5608007173616545  +/-  0.01


## OPEP/sTD-DFFT Jsc for any experimental PCE

In [32]:
#best_descriptors = optimize_eq('sTD-DFT', df, 'ExperimentalJsc')
best_descriptors = ['sTDDFTdeltaLUMO', 'sTDDFTsinglepointenergy', 'donsTDDFTdipolemoment', 'Planarity', 'donsTDDFTabsFOM', 'sTDDFTAccElectrodonating']
# + donsTDDFTdipolemoment
# - sTDDFTdeltaHOMO
LassoJsc_sTD= evaluate_model(best_descriptors, df['ExperimentalJsc'], df, cf=True)

The average intercept from 5 folds is  -25.045804857222894
The average coefficients of the 5 folds is  [('sTDDFTdeltaLUMO', -6.121261566409542), ('sTDDFTsinglepointenergy', -0.0011123741296276564), ('donsTDDFTdipolemoment', 0.32029430099411405), ('Planarity', -2.641717261116897), ('donsTDDFTabsFOM', 0.12393008889760833), ('sTDDFTAccElectrodonating', 4.746047823292268)]
The 5-fold cross-validated RMSE of this model is,  3.7685  +/-  0.0348
The 5-fold cross-validated MAE of the average  2.76  +/-  0.024
The average r^2 value is  0.5485638689225891  +/-  0.008


## OPEP/sTD-DFT FF Expanded for experimental PCE >9%

In [33]:
#best_descriptors =optimize_eq('sTD-DFT', df_sTD_highPCE, 'ExperimentalFF')
best_descriptors = ['sTDDFTsinglepointenergy', 'Planarity', 'donsTDDFTabsFOM', 'sTDDFTDonElectrodonating', 'sTDDFTdonfirstengtranswavenumber', 'sTDDFTaccfirstengtranswavenumber']
# + sTDDFTDonElectrodonating
# + sTDDFTdonfirstengtranswavenumber
# + sTDDFTaccfirstengtranswavenumber
# - donsTDDFTHOMO
# - sTDDFTabsFOM
# - sTDDFTAccChemHard
LassoFF_sTD_highPCE_all= evaluate_model(best_descriptors, df_sTD_highPCE['ExperimentalFF'], df_sTD_highPCE, cf=True)

The average intercept from 5 folds is  -28.692375076922303
The average coefficients of the 5 folds is  [('sTDDFTsinglepointenergy', -0.0005210546424254132), ('Planarity', -1.6326145999021549), ('donsTDDFTabsFOM', 0.1930491698473195), ('sTDDFTDonElectrodonating', 4.95410204883469), ('sTDDFTdonfirstengtranswavenumber', 0.002504918653491752), ('sTDDFTaccfirstengtranswavenumber', 0.0007897688798124622)]
The 5-fold cross-validated RMSE of this model is,  4.4713  +/-  0.01352
The 5-fold cross-validated MAE of the average  3.459  +/-  0.016
The average r^2 value is  0.3019946083855728  +/-  0.004


## OPEP/sTD-DFT FF for PCE >9%

In [34]:
#best_descriptors = optimize_eq('sTD-DFT', df_highPCE, 'ExperimentalFF')
best_descriptors = ['donsTDDFTdipolemoment', 'sTDDFTsummedoscs', 'donsTDDFTabsFOM', 'sTDDFTAccChemHard', 'sTDDFTdonlowestengtranswavenumber', 'sTDDFTaccfirstengtranswavenumber']
# Same as before.
LassoFF_sTD_highPCE= evaluate_model(best_descriptors, df_highPCE['ExperimentalFF'], df_highPCE, cf=True)

The average intercept from 5 folds is  189.59446308598882
The average coefficients of the 5 folds is  [('donsTDDFTdipolemoment', -2.6546526954880734), ('sTDDFTsummedoscs', 1.7234008292956733), ('donsTDDFTabsFOM', 0.22922317578275608), ('sTDDFTAccChemHard', 15.547672374232496), ('sTDDFTdonlowestengtranswavenumber', -0.008238859512288865), ('sTDDFTaccfirstengtranswavenumber', -0.0013526125599570748)]
The 5-fold cross-validated RMSE of this model is,  4.464  +/-  0.03558
The 5-fold cross-validated MAE of the average  3.497  +/-  0.036
The average r^2 value is  0.3159473776944752  +/-  0.011


## OPEP/sTD-DFT FF Expanded for any experimental PCE

In [35]:
#best_descriptors = optimize_eq('sTD-DFT', df_sTD, 'ExperimentalFF')
best_descriptors = ['donsTDDFTdipolemoment', 'sTDDFTsummedoscs', 'donsTDDFTabsFOM', 'sTDDFTAccChemHard', 'sTDDFTdonlowestengtranswavenumber', 'sTDDFTaccfirstengtranswavenumber']
# All different.
LassoFF_sTD_all = evaluate_model(best_descriptors, df_sTD['ExperimentalFF'], df_sTD, cf=True)

The average intercept from 5 folds is  -34.9819505282814
The average coefficients of the 5 folds is  [('donsTDDFTdipolemoment', 0.09647602504057687), ('sTDDFTsummedoscs', 4.024364310179203), ('donsTDDFTabsFOM', 0.02896514416800582), ('sTDDFTAccChemHard', 21.278860434412636), ('sTDDFTdonlowestengtranswavenumber', 0.000632252173892505), ('sTDDFTaccfirstengtranswavenumber', -0.0011851163323829254)]
The 5-fold cross-validated RMSE of this model is,  6.0857  +/-  0.01851
The 5-fold cross-validated MAE of the average  4.781  +/-  0.029
The average r^2 value is  0.15544780028291127  +/-  0.005


## OPEP/sTD-DFT FF for any experimental PCE

In [36]:
#best_descriptors = optimize_eq('sTD-DFT', df, 'ExperimentalFF')
best_descriptors = ['sTDDFTdeltaLUMO', 'donsTDDFTsinglepointenergy', 'sTDDFTHOMOoffset', 'sTDDFTabsFOM', 'donsTDDFTabsFOM', 'sTDDFTdonfirstengtranswavenumber']
# + sTDDFTabsFOM
# + donsTDDFTabsFOM
# + sTDDFTdonfirstengtranswavenumber
# - sTDDFTsinglepointenergy
# - Planarity
# - donsTDDFTsummedoscs
LassoFF_sTD= evaluate_model(best_descriptors, df['ExperimentalFF'], df, cf=True)

The average intercept from 5 folds is  -25.536341580303866
The average coefficients of the 5 folds is  [('sTDDFTdeltaLUMO', -14.15127724372849), ('donsTDDFTsinglepointenergy', -0.003084506780745354), ('sTDDFTHOMOoffset', 21.0103824966412), ('sTDDFTabsFOM', 0.536368766591085), ('donsTDDFTabsFOM', -0.4943809638580208), ('sTDDFTdonfirstengtranswavenumber', 0.0026463542595993052)]
The 5-fold cross-validated RMSE of this model is,  5.9685  +/-  0.03388
The 5-fold cross-validated MAE of the average  4.708  +/-  0.053
The average r^2 value is  0.24062983300554333  +/-  0.009


# Histogram of experimental fill factors

In [ ]:
import scipy.stats as stats
import math

SMALL_SIZE = 10
MEDIUM_SIZE = 12
BIGGER_SIZE = 15

#plt.style.use('seaborn')
pal = sns.color_palette("hls", 8)
pal = pal.as_hex()

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes

x = df_sTD.ExperimentalFF
mean = np.mean(df_sTD.ExperimentalFF)
std = np.std(df_sTD.ExperimentalFF)

fig, ax1 = plt.subplots(figsize=(10, 6))
n, bins, patches = ax1.hist(x, bins = 20, density = False, color = 'darkgray')

ax1.axvline(65, ls = '--', color = 'teal', lw = 2)
ax1.axvline(70, ls = '--', color = 'r', lw = 2)
ax1.axis([40, 80, 0, 15])

ax1.text(61, 13, "Scharber\n FF = 65%", fontsize = 15, verticalalignment = 'center', ha = 'center', color = 'teal')
ax1.text(74, 13, "Imamura\n FF = 70%", fontsize = 15, verticalalignment = 'center', ha = 'center', color = 'r')

ax1.set(xlabel = 'Fill Factor (%)', ylabel = 'Number of occurences')
plt.tight_layout()
plt.savefig(os.path.join(tgt_dir, 'FF_histogram.pdf'))
plt.show()

# Comparing cross-validated models

In [38]:
class SMWrapper(BaseEstimator, RegressorMixin):
    """ A universal sklearn-style wrapper for statsmodels regressors """
    def __init__(self, model_class, fit_intercept=True):
        self.model_class = model_class
        self.fit_intercept = fit_intercept
    def fit(self, X, y):
        if self.fit_intercept:
            X = sm.add_constant(X)
        self.model_ = self.model_class(y, X)
        self.results_ = self.model_.fit()
    def predict(self, X):
        if self.fit_intercept:
            X = sm.add_constant(X)
        return self.results_.predict(X)
    


def CrossValModels(predicted, target):
    # predicted is the list of predicted values. Ex: df['ScharberPCEtotal'] or PerrisPCEcalc
    # target is the target experimental value. Ex: df['ExperimentalPCE']

    X = predicted
    y = target
    score = cross_val_score(SMWrapper(sm.OLS), X, y, cv=5, scoring = 'r2')
    SEM_r2 = round((score.std() / math.sqrt(5)) , 3)
    #r2 = str(round(score.mean(), 3)) + ' +/- ' +  str(SEM_r2)
    r2 = round(float(score.mean()), 3)
    
    score = cross_val_score(SMWrapper(sm.OLS), X, y, cv=5, scoring = 'neg_mean_absolute_error')
    score = np.multiply(score, -1)
    SEM_MAE = round((score.std() / math.sqrt(5)) , 2)
    #MAE = str(round(score.mean(), 2)) + ' +/- ' +  str(SEM_MAE)
    MAE = round(score.mean(), 2) 
    
    score = cross_val_score(SMWrapper(sm.OLS), X, y, cv=5, scoring = 'neg_mean_squared_error')
    score = np.sqrt(score * -1)
    SEM_RMSE = round((score.std() / math.sqrt(5)) , 2)
    #RMSE = str(round(score.mean(), 2)) + ' +/- ' +  str(SEM_RMSE)
    RMSE = round(score.mean(), 2)
    
    MAPE_scorer = make_scorer(mean_absolute_percentage_error)
    score = cross_val_score(SMWrapper(sm.OLS), X, y, cv=5, scoring = MAPE_scorer)
    score = score * 100
    SEM_MAPE = round((score.std() / math.sqrt(5)) , 2)
    MAPE = round(score.mean(), 2)

    return round(r2,2), SEM_r2, MAE, SEM_MAE, RMSE, SEM_RMSE, MAPE, SEM_MAPE

In [39]:
# Table 1 is using only DFT data
OPEP_B3LYP_PCE_Calc_values = []
OPEP_B3LYP_PCE_FF65_values = []
OPEP_B3LYP_PCE_FF67_values = []
OPEP_B3LYP_PCE_FF70_values = []

for i in range(len(LassoFF[-1])):
    # calculating PCE using models to predict Voc, Jsc, and FF
    OPEP_B3LYP_PCE_Calc_values.append(LassoVoc[-1][i] * (LassoFF[-1][i]/100) * LassoJsc[-1][i])
    # examining if setting the FF to various constants is better than predicting it for calculating PCE
    OPEP_B3LYP_PCE_FF65_values.append(LassoVoc[-1][i] * 0.65 * LassoJsc[-1][i])
    OPEP_B3LYP_PCE_FF67_values.append(LassoVoc[-1][i] * 0.67 * LassoJsc[-1][i])
    OPEP_B3LYP_PCE_FF70_values.append(LassoVoc[-1][i] * 0.70 * LassoJsc[-1][i])

OPEP_B3LYP_PCE_Calc = CrossValModels(OPEP_B3LYP_PCE_Calc_values, df['ExperimentalPCE'])
OPEP_B3LYP_PCE_FF65 = CrossValModels(OPEP_B3LYP_PCE_FF65_values, df['ExperimentalPCE'])
OPEP_B3LYP_PCE_FF67 = CrossValModels(OPEP_B3LYP_PCE_FF67_values, df['ExperimentalPCE'])
OPEP_B3LYP_PCE_FF70 = CrossValModels(OPEP_B3LYP_PCE_FF70_values, df['ExperimentalPCE'])

SchPCEtot = CrossValModels(df['ScharberPCEtotal'], df['ExperimentalPCE'])
SchPCEdon = CrossValModels(df['ScharberPCEdon'], df['ExperimentalPCE'])
SchPCEacc = CrossValModels(df['ScharberPCEacc'], df['ExperimentalPCE'])
SchPCElowestoptbg = CrossValModels(df['ScharberPCElowestoptbg'], df['ExperimentalPCE'])
ImamuraPCEtot = CrossValModels(df['ImamuraPCEtotal'], df['ExperimentalPCE'])
ImamuraPCEacc = CrossValModels(df['ImamuraPCEacc'], df['ExperimentalPCE'])
ImamuraPCEdon = CrossValModels(df['ImamuraPCEdon'], df['ExperimentalPCE'])
AlharbiPCEDon = CrossValModels(df['AlharbiPCEDon'], df['ExperimentalPCE'])
AlharbiPCEAcc = CrossValModels(df['AlharbiPCEAcc'], df['ExperimentalPCE'])
OPEP_B3LYP_PCE_pred= LassoPCE[0:8]


list_Table1 = [
    SchPCEtot, SchPCEdon, SchPCEacc, SchPCElowestoptbg, ImamuraPCEtot, ImamuraPCEacc, ImamuraPCEdon,
    AlharbiPCEDon, AlharbiPCEAcc, OPEP_B3LYP_PCE_Calc, OPEP_B3LYP_PCE_pred]
Table1 = pd.DataFrame(
    list_Table1,
    columns=['r^2', '+/- r^2 SEM', 'MAE', '+/- MAE SEM','RMSE', '+/- RMSE SEM', 'MAPE', '+/- MAPE SEM'],
    index=[
        'SchPCEtot', 'SchPCEdon', 'SchPCEacc', 'SchPCElowestoptbg',
        'ImamuraPCEtot', 'ImamuraPCEacc', 'ImamuraPCEdon', 'AlharbiPCEDon', 'AlharbiPCEAcc',
        'OPEP/B3LYP Calculated PCE', 'OPEP/B3LYP Predicted PCE'])
Table1.sort_values(by=["RMSE"], inplace=True, ascending = True)
display(Table1)

#Table 2 is using only DFT data with experimental PCE > 9%
OPEP_B3LYP_PCE_Calc_highPCE_values = []
OPEP_B3LYP_PCE_FF65_highPCE_values = []
OPEP_B3LYP_PCE_FF67_highPCE_values = []
OPEP_B3LYP_PCE_FF70_highPCE_values = []
for i in range(len(LassoFF_highPCE[-1])):
    OPEP_B3LYP_PCE_Calc_highPCE_values.append(LassoVoc_highPCE[-1][i] * (LassoFF_highPCE[-1][i]/100) * LassoJsc_highPCE[-1][i])
    OPEP_B3LYP_PCE_FF65_highPCE_values.append(LassoVoc_highPCE[-1][i] * 0.65 * LassoJsc_highPCE[-1][i])
    OPEP_B3LYP_PCE_FF67_highPCE_values.append(LassoVoc_highPCE[-1][i] * 0.67 * LassoJsc_highPCE[-1][i])
    OPEP_B3LYP_PCE_FF70_highPCE_values.append(LassoVoc_highPCE[-1][i] * 0.70 * LassoJsc_highPCE[-1][i])
OPEP_B3LYP_PCE_Calc_highPCE = CrossValModels(OPEP_B3LYP_PCE_Calc_highPCE_values, df_highPCE['ExperimentalPCE'])
OPEP_B3LYP_PCE_FF65_highPCE = CrossValModels(OPEP_B3LYP_PCE_FF65_highPCE_values, df_highPCE['ExperimentalPCE'])
OPEP_B3LYP_PCE_FF67_highPCE = CrossValModels(OPEP_B3LYP_PCE_FF67_highPCE_values, df_highPCE['ExperimentalPCE'])
OPEP_B3LYP_PCE_FF70_highPCE = CrossValModels(OPEP_B3LYP_PCE_FF70_highPCE_values, df_highPCE['ExperimentalPCE'])

SchPCEtot_highPCE = CrossValModels(df_highPCE['ScharberPCEtotal'], df_highPCE['ExperimentalPCE'])
SchPCEdon_highPCE = CrossValModels(df_highPCE['ScharberPCEdon'], df_highPCE['ExperimentalPCE'])
SchPCEacc_highPCE = CrossValModels(df_highPCE['ScharberPCEacc'], df_highPCE['ExperimentalPCE'])
SchPCElowestoptbg_highPCE = CrossValModels(df_highPCE['ScharberPCElowestoptbg'], df_highPCE['ExperimentalPCE'])
ImamuraPCEtot_highPCE = CrossValModels(df_highPCE['ImamuraPCEtotal'], df_highPCE['ExperimentalPCE'])
ImamuraPCEacc_highPCE = CrossValModels(df_highPCE['ImamuraPCEacc'], df_highPCE['ExperimentalPCE'])
ImamuraPCEdon_highPCE = CrossValModels(df_highPCE['ImamuraPCEdon'], df_highPCE['ExperimentalPCE'])
AlharbiPCEDon_highPCE = CrossValModels(df_highPCE['AlharbiPCEDon'], df_highPCE['ExperimentalPCE'])
AlharbiPCEAcc_highPCE = CrossValModels(df_highPCE['AlharbiPCEAcc'], df_highPCE['ExperimentalPCE'])
OPEP_B3LYP_PCE_pred_highPCE = LassoPCE_highPCE[0:8]

list_Table2 = [
    OPEP_B3LYP_PCE_pred_highPCE, SchPCEtot_highPCE, SchPCEdon_highPCE, SchPCEacc_highPCE, SchPCElowestoptbg_highPCE,
    ImamuraPCEtot_highPCE, ImamuraPCEacc_highPCE, ImamuraPCEdon_highPCE,
    AlharbiPCEDon_highPCE, AlharbiPCEAcc_highPCE, OPEP_B3LYP_PCE_Calc_highPCE]
Table2 = pd.DataFrame(
    list_Table2,
    columns=['r^2', '+/- r^2 SEM','MAE','+/- MAE SEM', 'RMSE', '+/- RMSE SEM', 'MAPE', '+/- MAPE SEM'],
    index=[
        'OPEP/B3LYP PCE pred highPCE', 'SchPCEtot_highPCE', 'SchPCEdon_highPCE', 'SchPCEacc_highPCE', 'SchPCElowestoptbg_highPCE',
        'ImamuraPCEtot_highPCE', 'ImamuraPCEacc_highPCE', 'ImamuraPCEdon_highPCE',
        'AlharbiPCEDon_highPCE', 'AlharbiPCEAcc_highPCE', 'OPEP/B3LYP PCE calc highPCE'])
Table2.sort_values(by=["RMSE"], inplace=True, ascending = True)
display(Table2)

#Table 3 is using only sTD-DFT data with same experimental acceptor-donor pairs as Tables 1 and 2
OPEP_sTDDFT_PCE_calc_values = []
OPEP_sTDDFT_PCE_FF70_values = []

for i in range(len(LassoFF_sTD[-1])):
    OPEP_sTDDFT_PCE_calc_values.append(LassoVoc_sTD[-1][i] * (LassoFF_sTD[-1][i]/100) * LassoJsc_sTD[-1][i])
    OPEP_sTDDFT_PCE_FF70_values.append(LassoVoc_sTD[-1][i] * 0.70 * LassoJsc_sTD[-1][i])
OPEP_sTDDFT_PCE_calc = CrossValModels(OPEP_sTDDFT_PCE_calc_values, df['ExperimentalPCE'])
OPEP_sTDDFT_PCE_FF70 = CrossValModels(OPEP_sTDDFT_PCE_FF70_values, df['ExperimentalPCE'])

sTDDFTSchPCETot = CrossValModels(df['sTDDFTScharberPCETot'], df['ExperimentalPCE'])
sTDDFTSchPCEAcc = CrossValModels(df['sTDDFTScharberPCEAcc'], df['ExperimentalPCE'])
sTDDFTSchPCEDon = CrossValModels(df['sTDDFTScharberPCEDon'], df['ExperimentalPCE'])
OPEP_sTDDFT_PCE_pred = LassoPCE_sTD[0:8]

list_Table3 = [sTDDFTSchPCETot, sTDDFTSchPCEAcc, sTDDFTSchPCEDon, OPEP_sTDDFT_PCE_pred, OPEP_sTDDFT_PCE_calc, OPEP_sTDDFT_PCE_FF70]
Table3 = pd.DataFrame(
    list_Table3,
    columns=['r^2', '+/- r^2 SEM','MAE', '+/- MAE SEM','RMSE', '+/- RMSE SEM', 'MAPE', '+/- MAPE SEM'],
    index=['sTDDFTSchPCETot', 'sTDDFTSchPCEAcc', 'sTDDFTSchPCEDon', 'OPEP/sTDDFT PCE pred', 'OPEP/sTDDFT PCE calc', 'OPEP/sTDDFT PCE FF=70'])
Table3.sort_values(by=["RMSE"], inplace=True, ascending = True)
display(Table3)

#Table 4 is using only sTD-DFT data with same experimental acceptor-donor pairs as Tables 1 and 2, but with experimental PCE > 9%
OPEP_sTDDFT_PCE_calc_highPCE_values = []
OPEP_sTDDFT_PCE_FF70_highPCE_values = []
for i in range(len(LassoFF_sTD_highPCE[-1])):
    OPEP_sTDDFT_PCE_calc_highPCE_values.append(LassoVoc_sTD_highPCE[-1][i] * (LassoFF_sTD_highPCE[-1][i]/100) * LassoJsc_sTD_highPCE[-1][i])
    OPEP_sTDDFT_PCE_FF70_highPCE_values.append(LassoVoc_sTD_highPCE[-1][i] * 0.70 * LassoJsc_sTD_highPCE[-1][i])
OPEP_sTDDFT_PCE_calc_highPCE = CrossValModels(OPEP_sTDDFT_PCE_calc_highPCE_values, df_highPCE['ExperimentalPCE'])
OPEP_sTDDFT_PCE_FF70_highPCE = CrossValModels(OPEP_sTDDFT_PCE_FF70_highPCE_values, df_highPCE['ExperimentalPCE'])

sTDDFTSchPCETot_highPCE = CrossValModels(df_highPCE['sTDDFTScharberPCETot'], df_highPCE['ExperimentalPCE'])
sTDDFTSchPCEAcc_highPCE = CrossValModels(df_highPCE['sTDDFTScharberPCEAcc'], df_highPCE['ExperimentalPCE'])
sTDDFTSchPCEDon_highPCE = CrossValModels(df_highPCE['sTDDFTScharberPCEDon'], df_highPCE['ExperimentalPCE'])
OPEP_sTDDFT_PCE_pred_highPCE = LassoPCE_sTD_highPCE[0:8]

list_Table4 = [sTDDFTSchPCETot_highPCE, sTDDFTSchPCEAcc_highPCE, sTDDFTSchPCEDon_highPCE, OPEP_sTDDFT_PCE_pred_highPCE, OPEP_sTDDFT_PCE_calc_highPCE, OPEP_sTDDFT_PCE_FF70_highPCE]
Table4 = pd.DataFrame(list_Table4, columns = ['r^2', '+/- r^2 SEM','MAE', '+/- MAE SEM','RMSE', '+/- RMSE SEM', 'MAPE', '+/- MAPE SEM'], index = ['sTDDFTSchPCETot_highPCE', 'sTDDFTSchPCEAcc_highPCE', 'sTDDFTSchPCEDon_highPCE', 'OPEP/sTDDFT PCE pred highPCE', 'OPEP/sTDDFT PCE calc highPCE', 'OPEP/sTDDFT PCE FF=70 highPCE'])
Table4.sort_values(by=["RMSE"], inplace=True, ascending = True)
display(Table4)

#Table 5 is using only sTD-DFT data but with additional acceptor-donor pairs
OPEP_sTDDFT_PCE_calc_expanded_values = []
OPEP_sTDDFT_PCE_FF70_expanded_values = []
for i in range(len(LassoFF_sTD_all[-1])):
    OPEP_sTDDFT_PCE_calc_expanded_values.append(LassoVoc_sTD_all[-1][i] * (LassoFF_sTD_all[-1][i]/100) * LassoJsc_sTD_all[-1][i])
    OPEP_sTDDFT_PCE_FF70_expanded_values.append(LassoVoc_sTD_all[-1][i] * 0.70 * LassoJsc_sTD_all[-1][i])
OPEP_sTDDFT_PCE_calc_expanded = CrossValModels(OPEP_sTDDFT_PCE_calc_expanded_values, df_sTD['ExperimentalPCE'])
OPEP_sTDDFT_PCE_FF70_expanded = CrossValModels(OPEP_sTDDFT_PCE_FF70_expanded_values, df_sTD['ExperimentalPCE'])

sTDDFTSchPCETotall = CrossValModels(df_sTD['sTDDFTScharberPCETot'], df_sTD['ExperimentalPCE'])
sTDDFTSchPCEAccall = CrossValModels(df_sTD['sTDDFTScharberPCEAcc'], df_sTD['ExperimentalPCE'])
sTDDFTSchPCEDonall = CrossValModels(df_sTD['sTDDFTScharberPCEDon'], df_sTD['ExperimentalPCE'])
OPEP_sTDDFT_PCE_pred_expanded = LassoPCE_sTD_all[0:8]

list_Table5 = [sTDDFTSchPCETotall, sTDDFTSchPCEAccall, sTDDFTSchPCEDonall, OPEP_sTDDFT_PCE_pred_expanded, OPEP_sTDDFT_PCE_calc_expanded, OPEP_sTDDFT_PCE_FF70_expanded]
Table5 = pd.DataFrame(list_Table5, columns = ['r^2', '+/- r^2 SEM','MAE','+/- MAE SEM', 'RMSE', '+/- RMSE SEM', 'MAPE', '+/- MAPE SEM'], index = ['sTDDFTSchPCETotall', 'sTDDFTSchPCEAccall', 'sTDDFTSchPCEDonall', 'OPEP/sTDDFT Expanded PCE pred', 'OPEP/sTDDFT Expanded PCE calc', 'OPEP/sTDDFT Expanded PCE FF=70'])
Table5.sort_values(by=["RMSE"], inplace=True, ascending = True)
display(Table5)

#Table 6 is using only sTD-DFT data but with additional acceptor-donor pairs, and PCE > 9%
OPEP_sTDDFT_PCE_calc_expanded_highPCE_values = []
OPEP_sTDDFT_PCE_FF70_expanded_highPCE_values = []
for i in range(len(LassoFF_sTD_highPCE_all[-1])):
    OPEP_sTDDFT_PCE_calc_expanded_highPCE_values.append(LassoVoc_sTD_highPCE_all[-1][i] * (LassoFF_sTD_highPCE_all[-1][i]/100) * LassoJsc_sTD_highPCE_all[-1][i])
    OPEP_sTDDFT_PCE_FF70_expanded_highPCE_values.append(LassoVoc_sTD_highPCE_all[-1][i] * 0.70 * LassoJsc_sTD_highPCE_all[-1][i])
OPEP_sTDDFT_PCE_calc_expanded_highPCE = CrossValModels(OPEP_sTDDFT_PCE_calc_expanded_highPCE_values, df_sTD_highPCE['ExperimentalPCE'])
OPEP_sTDDFT_PCE_FF70_expanded_highPCE = CrossValModels(OPEP_sTDDFT_PCE_FF70_expanded_highPCE_values, df_sTD_highPCE['ExperimentalPCE'])

sTDDFTSchPCETotall_highPCE = CrossValModels(df_sTD_highPCE['sTDDFTScharberPCETot'], df_sTD_highPCE['ExperimentalPCE'])
sTDDFTSchPCEAccall_highPCE = CrossValModels(df_sTD_highPCE['sTDDFTScharberPCEAcc'], df_sTD_highPCE['ExperimentalPCE'])
sTDDFTSchPCEDonall_highPCE = CrossValModels(df_sTD_highPCE['sTDDFTScharberPCEDon'], df_sTD_highPCE['ExperimentalPCE'])
OPEP_sTDDFT_PCE_pred_expanded_highPCE = LassoPCE_sTD_highPCE_all[0:8]

list_Table6 = [sTDDFTSchPCETotall_highPCE, sTDDFTSchPCEAccall_highPCE, sTDDFTSchPCEDonall_highPCE, OPEP_sTDDFT_PCE_pred_expanded_highPCE, OPEP_sTDDFT_PCE_calc_expanded_highPCE, OPEP_sTDDFT_PCE_FF70_expanded_highPCE]
Table6 = pd.DataFrame(list_Table6, columns = ['r^2', '+/- r^2 SEM','MAE', '+/- MAE SEM','RMSE', '+/- RMSE SEM', 'MAPE', '+/- MAPE SEM'], index = ['sTDDFTSchPCETotall_highPCE', 'sTDDFTSchPCEAccall_highPCE', 'sTDDFTSchPCEDonall_highPCE', 'OPEP/sTDDFT PCE Expanded pred highPCE', 'OPEP/sTDDFT PCE Expanded calc highPCE', 'OPEP/sTDDFT PCE Expanded FF=70 highPCE'])
Table6.sort_values(by=["RMSE"], inplace=True, ascending = True)
display(Table6)




,r^2,+/- r^2 SEM,MAE,+/- MAE SEM,RMSE,+/- RMSE SEM,MAPE,+/- MAPE SEM
OPEP/B3LYP Predicted PCE,0.548,0.004,1.951,0.016,2.5562,0.01066,28.9343,0.28239
OPEP/B3LYP Calculated PCE,0.290,0.057,2.430,0.170,2.9900,0.17000,30.8100,6.30000
AlharbiPCEDon,-0.100,0.091,3.200,0.070,3.6900,0.08000,42.7100,7.33000
AlharbiPCEAcc,-0.120,0.082,3.130,0.190,3.7600,0.21000,46.1100,10.31000
SchPCEtot,-0.140,0.075,3.150,0.160,3.7800,0.19000,45.1100,9.31000
ImamuraPCEtot,-0.140,0.075,3.150,0.160,3.7800,0.19000,45.1100,9.31000
SchPCEdon,-0.150,0.067,3.250,0.100,3.7900,0.10000,44.5500,8.17000
ImamuraPCEdon,-0.150,0.067,3.250,0.100,3.7900,0.10000,44.5500,8.17000
SchPCEacc,-0.260,0.055,3.280,0.180,4.0000,0.21000,49.0600,10.07000
ImamuraPCEacc,-0.260,0.055,3.280,0.180,4.0000,0.21000,49.0600,10.07000


,r^2,+/- r^2 SEM,MAE,+/- MAE SEM,RMSE,+/- RMSE SEM,MAPE,+/- MAPE SEM
OPEP/B3LYP PCE pred highPCE,0.634,0.014,1.246,0.014,1.5431,0.02969,10.225,0.1524
OPEP/B3LYP PCE calc highPCE,0.570,0.065,1.320,0.150,1.6000,0.14000,10.890,1.1400
SchPCEacc_highPCE,0.050,0.060,2.120,0.140,2.4300,0.13000,17.570,1.2900
ImamuraPCEacc_highPCE,0.050,0.060,2.120,0.140,2.4300,0.13000,17.570,1.2900
SchPCElowestoptbg_highPCE,0.040,0.059,2.140,0.130,2.4400,0.13000,17.720,1.1100
SchPCEdon_highPCE,0.020,0.038,2.100,0.100,2.4700,0.09000,17.610,0.7500
ImamuraPCEdon_highPCE,0.020,0.038,2.100,0.100,2.4700,0.09000,17.610,0.7500
AlharbiPCEDon_highPCE,-0.010,0.141,2.020,0.080,2.4800,0.18000,17.000,0.5800
AlharbiPCEAcc_highPCE,-0.010,0.054,2.210,0.130,2.5000,0.12000,18.360,1.3100
SchPCEtot_highPCE,-0.080,0.039,2.320,0.080,2.5800,0.08000,19.310,0.7300


,r^2,+/- r^2 SEM,MAE,+/- MAE SEM,RMSE,+/- RMSE SEM,MAPE,+/- MAPE SEM
OPEP/sTDDFT PCE calc,0.400,0.054,2.170,0.10,2.7400,0.19000,30.3300,5.92000
OPEP/sTDDFT PCE FF=70,0.390,0.045,2.210,0.11,2.7800,0.17000,30.2100,6.09000
OPEP/sTDDFT PCE pred,0.455,0.010,2.078,0.03,2.8037,0.02614,30.0418,0.32939
sTDDFTSchPCEDon,-0.070,0.102,3.120,0.03,3.6300,0.06000,41.3900,6.66000
sTDDFTSchPCETot,-0.080,0.073,3.070,0.11,3.6800,0.12000,42.7500,7.40000
sTDDFTSchPCEAcc,-0.190,0.060,3.220,0.17,3.8800,0.15000,47.0000,9.23000


,r^2,+/- r^2 SEM,MAE,+/- MAE SEM,RMSE,+/- RMSE SEM,MAPE,+/- MAPE SEM
OPEP/sTDDFT PCE calc highPCE,0.540,0.071,1.350,0.150,1.6700,0.1700,11.1600,1.35000
OPEP/sTDDFT PCE pred highPCE,0.567,0.019,1.278,0.015,1.6773,0.0377,10.9467,0.11257
OPEP/sTDDFT PCE FF=70 highPCE,0.500,0.051,1.480,0.110,1.7500,0.1300,12.3900,0.98000
sTDDFTSchPCEAcc_highPCE,-0.080,0.028,2.320,0.090,2.5900,0.0900,19.3900,0.76000
sTDDFTSchPCEDon_highPCE,-0.090,0.061,2.240,0.070,2.5900,0.0900,18.8200,0.70000
sTDDFTSchPCETot_highPCE,-0.100,0.046,2.340,0.070,2.6000,0.0700,19.5900,0.71000


,r^2,+/- r^2 SEM,MAE,+/- MAE SEM,RMSE,+/- RMSE SEM,MAPE,+/- MAPE SEM
OPEP/sTDDFT Expanded PCE calc,0.400,0.057,2.020,0.110,2.5600,0.15000,27.9700,3.72000
OPEP/sTDDFT Expanded PCE pred,0.475,0.002,2.002,0.015,2.5894,0.00591,27.4417,0.27855
OPEP/sTDDFT Expanded PCE FF=70,0.380,0.062,2.050,0.110,2.6000,0.17000,28.0700,3.77000
sTDDFTSchPCETotall,-0.140,0.073,2.970,0.240,3.5500,0.21000,40.6400,6.62000
sTDDFTSchPCEDonall,-0.150,0.060,3.000,0.210,3.5600,0.18000,39.8900,6.15000
sTDDFTSchPCEAccall,-0.190,0.083,2.990,0.280,3.6300,0.25000,42.3700,7.53000


,r^2,+/- r^2 SEM,MAE,+/- MAE SEM,RMSE,+/- RMSE SEM,MAPE,+/- MAPE SEM
OPEP/sTDDFT PCE Expanded pred highPCE,0.563,0.013,1.28,0.018,1.5807,0.02367,10.7877,0.12097
OPEP/sTDDFT PCE Expanded calc highPCE,0.390,0.060,1.43,0.130,1.7100,0.12000,12.0700,0.98000
OPEP/sTDDFT PCE Expanded FF=70 highPCE,0.290,0.059,1.55,0.140,1.8500,0.14000,13.2500,1.07000
sTDDFTSchPCEAccall_highPCE,-0.170,0.102,2.12,0.190,2.3800,0.20000,17.7400,1.08000
sTDDFTSchPCETotall_highPCE,-0.200,0.108,2.15,0.190,2.4100,0.20000,17.9400,1.01000
sTDDFTSchPCEDonall_highPCE,-0.230,0.138,2.14,0.180,2.4200,0.19000,17.8900,0.95000


# Comparing models for FF, Jsc, and Voc

In [40]:
#Table 7 is comparing all models for Fill Factor (FF) with high PCE
ScharberFF_highPCE = CrossValModels(df_highPCE['ScharberFF'], df_highPCE['ExperimentalFF'])
AlharbiFFDon_highPCE = CrossValModels(df_highPCE['AlharbiFFDon'], df_highPCE['ExperimentalFF'])
AlharbiFFAcc_highPCE = CrossValModels(df_highPCE['AlharbiFFAcc'], df_highPCE['ExperimentalFF'])
ImamuraFF_highPCE = CrossValModels(df_highPCE['ImamuraFF'], df_highPCE['ExperimentalFF'])
OPEP_B3LYP_FF_highPCE = LassoFF_highPCE[0:8]
OPEP_sTDDFT_FF_highPCE = LassoFF_sTD_highPCE[0:8]
OPEP_sTDDFT_Expanded_FF_highPCE = LassoFF_sTD_highPCE_all[0:8]

list_Table7 = [ScharberFF_highPCE, AlharbiFFDon_highPCE, AlharbiFFAcc_highPCE, ImamuraFF_highPCE, OPEP_B3LYP_FF_highPCE, OPEP_sTDDFT_FF_highPCE, OPEP_sTDDFT_Expanded_FF_highPCE]
Table7 = pd.DataFrame(list_Table7, columns = ['r^2', '+/- r^2 SEM','MAE', '+/- MAE SEM','RMSE', '+/- RMSE SEM', 'MAPE', '+/- MAPE SEM'], index = ['ScharberFF_highPCE', 'AlharbiFFDon_highPCE', 'AlharbiFFAcc_highPCE', 'ImamuraFF_highPCE', 'OPEP/B3LYP FF highPCE', 'OPEP/sTDDFT FF highPCE', 'OPEP/sTDDFT Expanded FF highPCE'])
Table7.sort_values(by=["RMSE"], inplace=True, ascending = True)
display(Table7)


#Table 7b is comparing all models for Fill Factor (FF) 
ScharberFF = CrossValModels(df['ScharberFF'], df['ExperimentalFF'])
AlharbiFFDon = CrossValModels(df['AlharbiFFDon'], df['ExperimentalFF'])
AlharbiFFAcc = CrossValModels(df['AlharbiFFAcc'], df['ExperimentalFF'])
ImamuraFF = CrossValModels(df['ImamuraFF'], df['ExperimentalFF'])
OPEP_B3LYP_FF = LassoFF[0:8]
OPEP_sTDDFT_FF = LassoFF_sTD[0:8]
OPEP_sTDDFT_Expanded_FF = LassoFF_sTD_all[0:8]

list_Table7b = [ScharberFF, AlharbiFFDon, AlharbiFFAcc, ImamuraFF, OPEP_B3LYP_FF, OPEP_sTDDFT_FF, OPEP_sTDDFT_Expanded_FF]
Table7b = pd.DataFrame(list_Table7b, columns = ['r^2', '+/- r^2 SEM','MAE', '+/- MAE SEM','RMSE', '+/- RMSE SEM', 'MAPE', '+/- MAPE SEM'], index = ['ScharberFF', 'AlharbiFFDon', 'AlharbiFFAcc', 'ImamuraFF', 'OPEP/B3LYP FF', 'OPEP/sTDDFT FF', 'OPEP/sTDDFT Expanded FF'])
Table7b.sort_values(by=["RMSE"], inplace=True, ascending = True)
display(Table7b)


#Table 8 is comparing all models for short circuit current (Jsc) for high PCE
ScharberJscAcc_highPCE = CrossValModels(df_highPCE['ScharberJscAcc'], df_highPCE['ExperimentalJsc'])
ImamuraTotalJsc_highPCE = CrossValModels(df_highPCE['ImamuraTotalJsc'], df_highPCE['ExperimentalJsc'])
ImamuraJscAcc_highPCE = CrossValModels(df_highPCE['ImamuraJscAcc'], df_highPCE['ExperimentalJsc'])
ImamuraJscDon_highPCE = CrossValModels(df_highPCE['ImamuraJscDon'], df_highPCE['ExperimentalJsc'])
ScharberTotalJsc_highPCE = CrossValModels(df_highPCE['ScharberTotalJsc'], df_highPCE['ExperimentalJsc'])
ScharberJscDon_highPCE = CrossValModels(df_highPCE['ScharberJscDon'], df_highPCE['ExperimentalJsc'])
ScharberJsclowestoptbg_highPCE = CrossValModels(df_highPCE['ScharberJsclowestoptbg'], df_highPCE['ExperimentalJsc'])
OPEP_B3LYP_Jsc_highPCE = LassoJsc_highPCE[0:8]
OPEP_sTDDFT_Jsc_highPCE = LassoJsc_sTD_highPCE[0:8]
OPEP_sTDDFT_Expanded_Jsc_highPCE = LassoJsc_sTD_highPCE_all[0:8]

list_Table8 = [ScharberJscAcc_highPCE, ImamuraTotalJsc_highPCE, ImamuraJscAcc_highPCE, ImamuraJscDon_highPCE, ScharberTotalJsc_highPCE, ScharberJscDon_highPCE, ScharberJsclowestoptbg_highPCE, OPEP_B3LYP_Jsc_highPCE, OPEP_sTDDFT_Jsc_highPCE, OPEP_sTDDFT_Expanded_Jsc_highPCE]
Table8 = pd.DataFrame(list_Table8, columns = ['r^2', '+/- r^2 SEM','MAE', '+/- MAE SEM','RMSE', '+/- RMSE SEM', 'MAPE', '+/- MAPE SEM'], index = ['ScharberJscAcc_highPCE', 'ImamuraTotalJsc_highPCE', 'ImamuraJscAcc_highPCE', 'ImamuraJscDon_highPCE', 'ScharberTotalJsc_highPCE', 'ScharberJscDon_highPCE', 'ScharberJsclowestoptbg_highPCE', 'OPEP/B3LYP Jsc highPCE', 'OPEP/sTDDFT Jsc highPCE', 'OPEP/sTDDFT Expanded Jsc highPCE '])
Table8.sort_values(by=["RMSE"], inplace=True, ascending = True)
display(Table8)

#Table 8b is comparing all models for short circuit current (Jsc)
ScharberJscAcc = CrossValModels(df['ScharberJscAcc'], df['ExperimentalJsc'])
ImamuraTotalJsc = CrossValModels(df['ImamuraTotalJsc'], df['ExperimentalJsc'])
ImamuraJscAcc = CrossValModels(df['ImamuraJscAcc'], df['ExperimentalJsc'])
ImamuraJscDon = CrossValModels(df['ImamuraJscDon'], df['ExperimentalJsc'])
ScharberTotalJsc = CrossValModels(df['ScharberTotalJsc'], df['ExperimentalJsc'])
ScharberJscDon = CrossValModels(df['ScharberJscDon'], df['ExperimentalJsc'])
ScharberJsclowestoptbg = CrossValModels(df['ScharberJsclowestoptbg'], df['ExperimentalJsc'])
OPEP_B3LYP_Jsc = LassoJsc[0:8]
OPEP_sTDDFT_Jsc = LassoJsc_sTD[0:8]
OPEP_sTDDFT_Expanded_Jsc = LassoJsc_sTD_all[0:8]


list_Table8b = [ScharberJscAcc, ImamuraTotalJsc, ImamuraJscAcc, ImamuraJscDon, ScharberTotalJsc, ScharberJscDon, ScharberJsclowestoptbg, OPEP_B3LYP_Jsc, OPEP_sTDDFT_Jsc, OPEP_sTDDFT_Expanded_Jsc]
Table8b = pd.DataFrame(list_Table8b, columns = ['r^2', '+/- r^2 SEM','MAE', '+/- MAE SEM','RMSE', '+/- RMSE SEM', 'MAPE', '+/- MAPE SEM'], index = ['ScharberJscAcc', 'ImamuraTotalJsc', 'ImamuraJscAcc', 'ImamuraJscDon', 'ScharberTotalJsc', 'ScharberJscDon', 'ScharberJsclowestoptbg','OPEP/B3LYP Jsc', 'OPEP/sTDDFT Jsc', 'OPEP/sTDDFT Expanded Jsc'])
Table8b.sort_values(by=["RMSE"], inplace=True, ascending = True)
display(Table8b)

#Table 9 is comparing all models for open circuit voltage (Voc) for high PCE
AlharbiVocAcc_highPCE = CrossValModels(df_highPCE['AlharbiVocAcc'], df_highPCE['ExperimentalVoc'])
AlharbiVocDon_highPCE = CrossValModels(df_highPCE['AlharbiVocDon'], df_highPCE['ExperimentalVoc'])
ImamuraVoc_highPCE = CrossValModels(df_highPCE['ImamuraVoc'], df_highPCE['ExperimentalVoc'])
ScharberVoc_highPCE = CrossValModels(df_highPCE['ScharberVoc'], df_highPCE['ExperimentalVoc'])
OPEP_B3LYP_Voc_highPCE = LassoVoc_highPCE[0:8]
OPEP_sTDDFT_Voc_highPCE = LassoVoc_sTD_highPCE[0:8]
OPEP_sTDDFT_Expanded_Voc_highPCE = LassoVoc_sTD_highPCE_all[0:8]

list_Table9 = [AlharbiVocAcc_highPCE, AlharbiVocDon_highPCE, ImamuraVoc_highPCE,  ScharberVoc_highPCE, OPEP_B3LYP_Voc_highPCE, OPEP_sTDDFT_Voc_highPCE, OPEP_sTDDFT_Expanded_Voc_highPCE]
Table9 = pd.DataFrame(list_Table9, columns = ['r^2', '+/- r^2 SEM','MAE', '+/- MAE SEM','RMSE', '+/- RMSE SEM', 'MAPE', '+/- MAPE SEM'], index = ['AlharbiVocAcc_highPCE', 'AlharbiVocDon_highPCE', 'ImamuraVoc_highPCE', 'ScharberVoc_highPCE', 'OPEP/B3LYP Voc highPCE', 'OPEP/sTDDFT Voc highPCE', 'OPEP/sTDDFT Expanded Voc highPCE'])
Table9.sort_values(by=["RMSE"], inplace=True, ascending = True)
display(Table9)

#Table 9b is comparing all models for open circuit voltage (Voc)
AlharbiVocAcc = CrossValModels(df['AlharbiVocAcc'], df['ExperimentalVoc'])
AlharbiVocDon = CrossValModels(df['AlharbiVocDon'], df['ExperimentalVoc'])
ImamuraVoc = CrossValModels(df['ImamuraVoc'], df['ExperimentalVoc'])
ScharberVoc = CrossValModels(df['ScharberVoc'], df['ExperimentalVoc'])
OPEP_B3LYP_Voc = LassoVoc[0:8]
OPEP_sTDDFT_Voc = LassoVoc_sTD[0:8]
OPEP_sTDDFT_Expanded_Voc = LassoVoc_sTD_all[0:8]

list_Table9b = [AlharbiVocAcc, AlharbiVocDon, ImamuraVoc,  ScharberVoc, OPEP_B3LYP_Voc, OPEP_sTDDFT_Voc, OPEP_sTDDFT_Expanded_Voc]
Table9b = pd.DataFrame(list_Table9b, columns = ['r^2', '+/- r^2 SEM','MAE', '+/- MAE SEM','RMSE', '+/- RMSE SEM', 'MAPE', '+/- MAPE SEM'], index = ['AlharbiVocAcc', 'AlharbiVocDon', 'ImamuraVoc', 'ScharberVoc', 'OPEP/B3LYP Voc', 'OPEP/sTDDFT Voc', 'OPEP/sTDDFT Expanded Voc'])
Table9b.sort_values(by=["RMSE"], inplace=True, ascending = True)
display(Table9b)

,r^2,+/- r^2 SEM,MAE,+/- MAE SEM,RMSE,+/- RMSE SEM,MAPE,+/- MAPE SEM
OPEP/B3LYP FF highPCE,0.401,0.007,3.195,0.033,4.1773,0.02616,4.7820,0.03948
OPEP/sTDDFT FF highPCE,0.316,0.011,3.497,0.036,4.4640,0.03558,5.2446,0.04440
OPEP/sTDDFT Expanded FF highPCE,0.302,0.004,3.459,0.016,4.4713,0.01352,5.1961,0.02052
AlharbiFFDon_highPCE,-0.080,0.120,4.080,0.340,5.0600,0.46000,6.0800,0.65000
AlharbiFFAcc_highPCE,-0.250,0.092,4.470,0.300,5.4700,0.42000,6.6400,0.60000
ScharberFF_highPCE,-0.250,0.086,4.500,0.280,5.4800,0.43000,6.6900,0.57000
ImamuraFF_highPCE,-0.250,0.086,4.500,0.280,5.4800,0.43000,6.6900,0.57000


,r^2,+/- r^2 SEM,MAE,+/- MAE SEM,RMSE,+/- RMSE SEM,MAPE,+/- MAPE SEM
OPEP/B3LYP FF,0.307,0.005,4.388,0.049,5.7005,0.02069,6.8859,0.06727
OPEP/sTDDFT FF,0.241,0.009,4.708,0.053,5.9685,0.03388,7.4209,0.07036
OPEP/sTDDFT Expanded FF,0.155,0.005,4.781,0.029,6.0857,0.01851,7.5467,0.01993
ScharberFF,-0.150,0.102,5.620,0.430,6.9100,0.49000,8.8200,0.77000
ImamuraFF,-0.150,0.102,5.620,0.430,6.9100,0.49000,8.8200,0.77000
AlharbiFFDon,-0.170,0.105,5.680,0.450,6.9600,0.50000,8.9100,0.78000
AlharbiFFAcc,-0.180,0.097,5.710,0.390,6.9800,0.44000,8.9500,0.70000


,r^2,+/- r^2 SEM,MAE,+/- MAE SEM,RMSE,+/- RMSE SEM,MAPE,+/- MAPE SEM
OPEP/sTDDFT Expanded Jsc highPCE,0.712,0.013,1.461,0.027,1.8309,0.03930,7.1835,0.11834
OPEP/sTDDFT Jsc highPCE,0.668,0.009,1.523,0.028,2.0051,0.02801,7.1688,0.18181
OPEP/B3LYP Jsc highPCE,0.664,0.004,1.610,0.016,2.0187,0.01247,7.5226,0.07976
ScharberJscAcc_highPCE,-0.010,0.193,2.630,0.180,3.1300,0.20000,12.8400,1.40000
ImamuraJscAcc_highPCE,-0.010,0.193,2.630,0.180,3.1300,0.20000,12.8400,1.40000
ScharberJsclowestoptbg_highPCE,-0.060,0.206,2.730,0.170,3.2100,0.18000,13.3900,1.28000
ImamuraTotalJsc_highPCE,-0.220,0.196,2.910,0.180,3.4700,0.20000,14.6300,1.50000
ScharberTotalJsc_highPCE,-0.220,0.196,2.910,0.180,3.4700,0.20000,14.6300,1.50000
ImamuraJscDon_highPCE,-0.270,0.111,3.030,0.170,3.6300,0.23000,15.3800,1.33000
ScharberJscDon_highPCE,-0.270,0.111,3.030,0.170,3.6300,0.23000,15.3800,1.33000


,r^2,+/- r^2 SEM,MAE,+/- MAE SEM,RMSE,+/- RMSE SEM,MAPE,+/- MAPE SEM
OPEP/sTDDFT Expanded Jsc,0.561,0.010,2.508,0.031,3.4879,0.04085,18.9233,0.19372
OPEP/sTDDFT Jsc,0.549,0.008,2.760,0.024,3.7685,0.03480,20.8910,0.15886
OPEP/B3LYP Jsc,0.531,0.005,2.850,0.016,3.8404,0.02102,20.6411,0.16168
ImamuraTotalJsc,-0.060,0.070,4.310,0.290,5.2100,0.31000,31.4900,6.33000
ScharberTotalJsc,-0.060,0.070,4.310,0.290,5.2100,0.31000,31.4900,6.33000
ScharberJscAcc,-0.150,0.098,4.390,0.300,5.4300,0.41000,33.2000,6.81000
ImamuraJscAcc,-0.150,0.098,4.390,0.300,5.4300,0.41000,33.2000,6.81000
ScharberJsclowestoptbg,-0.180,0.111,4.410,0.300,5.4700,0.40000,33.1200,6.68000
ImamuraJscDon,-0.190,0.104,4.640,0.300,5.4800,0.24000,33.1400,5.78000
ScharberJscDon,-0.190,0.104,4.640,0.300,5.4800,0.24000,33.1400,5.78000


,r^2,+/- r^2 SEM,MAE,+/- MAE SEM,RMSE,+/- RMSE SEM,MAPE,+/- MAPE SEM
OPEP/sTDDFT Voc highPCE,0.599,0.018,0.047,0.002,0.0593,0.00134,5.6735,0.18455
OPEP/sTDDFT Expanded Voc highPCE,0.522,0.030,0.047,0.002,0.0611,0.00190,5.5202,0.20181
OPEP/B3LYP Voc highPCE,0.559,0.006,0.050,0.001,0.0622,0.00043,6.0311,0.07065
ImamuraVoc_highPCE,0.040,0.312,0.060,0.010,0.0700,0.01000,7.1900,0.64000
ScharberVoc_highPCE,0.040,0.312,0.060,0.010,0.0700,0.01000,7.1900,0.64000
AlharbiVocAcc_highPCE,-0.370,0.272,0.070,0.010,0.1000,0.01000,8.7400,0.65000
AlharbiVocDon_highPCE,-0.410,0.305,0.070,0.000,0.1000,0.01000,8.8000,0.43000


,r^2,+/- r^2 SEM,MAE,+/- MAE SEM,RMSE,+/- RMSE SEM,MAPE,+/- MAPE SEM
OPEP/sTDDFT Voc,0.629,0.021,0.051,0.001,0.0685,0.00196,6.3561,0.17224
OPEP/sTDDFT Expanded Voc,0.471,0.026,0.058,0.002,0.0765,0.00187,7.1026,0.20050
OPEP/B3LYP Voc,0.360,0.031,0.068,0.001,0.0900,0.00212,8.5725,0.09941
ImamuraVoc,0.190,0.126,0.070,0.010,0.1000,0.01000,8.9700,0.93000
ScharberVoc,0.190,0.126,0.070,0.010,0.1000,0.01000,8.9700,0.93000
AlharbiVocDon,-0.220,0.103,0.090,0.010,0.1200,0.01000,11.5000,0.72000
AlharbiVocAcc,-0.550,0.401,0.100,0.020,0.1400,0.03000,12.8100,2.81000


# Analysis Figures

In [ ]:
def make_barplots(param, ax, stat=None):
    '''
    Makes a barplot comparing OPEP, Scharber, Imamura, and Alharbi models

    Parameters
    ----------
    param: str
        options are 'FF', 'Jsc', 'Voc', or 'PCE'
    ax: str
        axes in figure to draw barplot on
    stat: None or str
        if None, MAE and RMSE will be plotted. If 'MAPE', mean absolute percent error (MAPE) will be plotted.
    '''    
    if param == 'FF':
        MAE = [ScharberFF_highPCE[2], ImamuraFF_highPCE[2], AlharbiFFDon_highPCE[2], OPEP_B3LYP_FF_highPCE[2], OPEP_B3LYP_FF_highPCE[2] ]
        SEM_MAE =[ScharberFF_highPCE[3], ImamuraFF_highPCE[3], AlharbiFFDon_highPCE[3], OPEP_B3LYP_FF_highPCE[3], OPEP_B3LYP_FF_highPCE[3] ]
        RMSE = [ScharberFF_highPCE[4], ImamuraFF_highPCE[4], AlharbiFFDon_highPCE[4], OPEP_B3LYP_FF_highPCE[4], OPEP_B3LYP_FF_highPCE[4] ]
        SEM_RMSE=[ScharberFF_highPCE[5], ImamuraFF_highPCE[5], AlharbiFFDon_highPCE[5], OPEP_B3LYP_FF_highPCE[5], OPEP_B3LYP_FF_highPCE[5] ]
        MAPE = [ScharberFF_highPCE[6], ImamuraFF_highPCE[6], AlharbiFFDon_highPCE[6], OPEP_B3LYP_FF_highPCE[6], OPEP_B3LYP_FF_highPCE[6]]
        SEM_MAPE =  [ScharberFF_highPCE[7], ImamuraFF_highPCE[7], AlharbiFFDon_highPCE[7], OPEP_B3LYP_FF_highPCE[7], OPEP_B3LYP_FF_highPCE[7]]
        labels = ['Scharber', 'Imamura', 'Alharbi', 'OPEP / B3LYP', 'OPEP / sTD-DFT']
        x = np.arange(5)
        
    elif param == 'Voc':
        MAE = [ScharberVoc_highPCE[2], ImamuraVoc_highPCE[2], AlharbiVocDon_highPCE[2], OPEP_B3LYP_Voc_highPCE[2], OPEP_sTDDFT_Voc_highPCE[2] ]
        SEM_MAE = [ScharberVoc_highPCE[3], ImamuraVoc_highPCE[3], AlharbiVocDon_highPCE[3], OPEP_B3LYP_Voc_highPCE[3], OPEP_sTDDFT_Voc_highPCE[3]]
        RMSE = [ScharberVoc_highPCE[4], ImamuraVoc_highPCE[4], AlharbiVocDon_highPCE[4], OPEP_B3LYP_Voc_highPCE[4], OPEP_sTDDFT_Voc_highPCE[4]]
        SEM_RMSE=[ScharberVoc_highPCE[5], ImamuraVoc_highPCE[5], AlharbiVocDon_highPCE[5], OPEP_B3LYP_Voc_highPCE[5], OPEP_sTDDFT_Voc_highPCE[5]]
        MAPE = [ScharberVoc_highPCE[6], ImamuraVoc_highPCE[6], AlharbiVocDon_highPCE[6], OPEP_B3LYP_Voc_highPCE[6], OPEP_sTDDFT_Voc_highPCE[6]]
        SEM_MAPE =  [ScharberVoc_highPCE[7], ImamuraVoc_highPCE[7], AlharbiVocDon_highPCE[7], OPEP_B3LYP_Voc_highPCE[7], OPEP_sTDDFT_Voc_highPCE[7]]
        labels = ['Scharber', 'Imamura', 'Alharbi', 'OPEP / B3LYP', 'OPEP / sTD-DFT']
        x = np.arange(5)
    
    elif param == 'Jsc':
        MAE = [ScharberJscDon_highPCE[2], ImamuraJscDon_highPCE[2], OPEP_B3LYP_Jsc_highPCE[2], OPEP_sTDDFT_Jsc_highPCE[2] ]
        SEM_MAE = [ScharberJscDon_highPCE[3], ImamuraJscDon_highPCE[3],  OPEP_B3LYP_Jsc_highPCE[3], OPEP_sTDDFT_Jsc_highPCE[3]]
        RMSE = [ScharberJscDon_highPCE[4], ImamuraJscDon_highPCE[4],  OPEP_B3LYP_Jsc_highPCE[4], OPEP_sTDDFT_Jsc_highPCE[4]]
        SEM_RMSE=[ScharberJscDon_highPCE[5], ImamuraJscDon_highPCE[5],  OPEP_B3LYP_Jsc_highPCE[5], OPEP_sTDDFT_Jsc_highPCE[5]]
        MAPE = [ScharberJscDon_highPCE[6], ImamuraJscDon_highPCE[6], OPEP_B3LYP_Jsc_highPCE[6], OPEP_sTDDFT_Jsc_highPCE[6]]
        SEM_MAPE =  [ScharberJscDon_highPCE[7], ImamuraJscDon_highPCE[7], OPEP_B3LYP_Jsc_highPCE[7], OPEP_sTDDFT_Jsc_highPCE[7]]
        labels = ['Scharber', 'Imamura', 'OPEP / B3LYP', 'OPEP / sTD-DFT']
        x = np.arange(4)
        
    elif param == 'PCE':
        MAE = [SchPCEdon_highPCE[2], ImamuraPCEdon_highPCE[2], AlharbiPCEDon_highPCE[2],OPEP_B3LYP_PCE_pred_highPCE[2], OPEP_sTDDFT_PCE_pred_highPCE[2] ]
        SEM_MAE = [SchPCEdon_highPCE[3], ImamuraPCEdon_highPCE[3], AlharbiPCEDon_highPCE[3], OPEP_B3LYP_PCE_pred_highPCE[3], OPEP_sTDDFT_PCE_pred_highPCE[3]]
        RMSE = [SchPCEdon_highPCE[4], ImamuraPCEdon_highPCE[4], AlharbiPCEDon_highPCE[4], OPEP_B3LYP_PCE_pred_highPCE[4], OPEP_sTDDFT_PCE_pred_highPCE[4]]
        SEM_RMSE=[SchPCEdon_highPCE[5], ImamuraPCEdon_highPCE[5], AlharbiPCEDon_highPCE[5], OPEP_B3LYP_PCE_pred_highPCE[5], OPEP_sTDDFT_PCE_pred_highPCE[5]]
        MAPE = [SchPCEdon_highPCE[6], ImamuraPCEdon_highPCE[6], AlharbiPCEDon_highPCE[6],OPEP_B3LYP_PCE_pred_highPCE[6], OPEP_sTDDFT_PCE_pred_highPCE[6]]
        SEM_MAPE =  [SchPCEdon_highPCE[7], ImamuraPCEdon_highPCE[7], AlharbiPCEDon_highPCE[7],OPEP_B3LYP_PCE_pred_highPCE[7], OPEP_sTDDFT_PCE_pred_highPCE[7]]
        labels = ['Scharber', 'Imamura', 'Modified Alharbi','OPEP / B3LYP', 'OPEP / sTD-DFT']
        x = np.arange(5)
        
    else:
        return "not a valid param"
    
    width = 0.3
    if stat =='MAPE':
        ax.bar(x, MAPE, width, yerr=SEM_MAPE, label = "MAPE", color = 'firebrick')
        ax.legend(['MAPE'], loc = 'upper right')
    else:
        ax.bar(x-width/2, MAE, width, yerr=SEM_MAE, label = "MAE", color = '#007f92')
        ax.bar(x+width/2, RMSE, width, yerr=SEM_RMSE, label = "RMSE", color = '#a1196b')
        ax.legend(['MAE', 'RMSE'], loc = 'upper right')

    ax.set_xticks(x)
    ax.set_xticklabels(labels, rotation = 45, ha = 'right')
    ax.set_title(param, weight = 'bold')

Creates a barplot of MAE and RMSE comparing the various models for FF, Voc, Jsc, and PCE for the models trained on high experimental PCE. Error bars are the standard error of the mean (SEM) of each model.

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2, figsize = (12, 12))

make_barplots("FF", ax1)
make_barplots("Voc", ax2)
make_barplots("Jsc", ax3)
make_barplots("PCE", ax4)

ax1.set_ylabel('%')
ax2.set_ylabel('V')
ax3.set_ylabel('$\mathregular{mA/cm^{2}}$')
ax4.set_ylabel('%')

plt.tight_layout()
plt.subplots_adjust(top = 0.93, wspace = 0.3, hspace = 0.6)
plt.savefig(os.path.join(tgt_dir, 'barplots_highPCE.png'), dpi=300)
plt.savefig(os.path.join(tgt_dir, 'barplots_highPCE.svg'), dpi=300)
plt.savefig(os.path.join(tgt_dir, 'barplots_highPCE.pdf'), dpi=300)
plt.show()

Creates a barplot of MAPE comparing the various models for FF, Voc, Jsc, and PCE for the models trained on high experimental PCE. Error bars are the standard error of the mean (SEM) of each model.

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2, figsize = (12, 12))

make_barplots("FF", ax1, stat = 'MAPE')
make_barplots("Voc", ax2, stat = 'MAPE')
make_barplots("Jsc", ax3, stat = 'MAPE')
make_barplots("PCE", ax4, stat = 'MAPE')

ax1.set_ylabel('%')
ax2.set_ylabel('%')
ax3.set_ylabel('%')
ax4.set_ylabel('%')

plt.subplots_adjust(top = 0.93, wspace = 0.3, hspace = 0.4)
plt.tight_layout()
plt.savefig(os.path.join(tgt_dir, 'barplots_MAPE_highPCE.png'), dpi=300)
plt.savefig(os.path.join(tgt_dir, 'barplots_MAPE_highPCE.svg'), dpi=300)
plt.savefig(os.path.join(tgt_dir, 'barplots_MAPE_highPCE.pdf'), dpi=300)

plt.show()

In [ ]:
def make_scatter_plot(ax, feature, target, xlabel, ylabel, df=False, xyline=False, trendline=False):
    '''
    Creates a scatter plot comparing the predicted values to experimental values

    Parameters
    ---------
    ax: str
        axes on figure to plot it on
    feature: str
        Model name for predictions, as written in the dataframe. Example: 'ScharberPCEdon'
    target: str
        Values to compare to for y-axis, as written in the dataframe. Example: 'ExperimentalPCE'
    xlabel: str
        Name of model. Options are: 'Scharber PCE', 'Alharbi PCE', 'Imamura PCE', 'OPEP / B3LYP', 'OPEP / sTD-DFT'
    ylabel: str
        Label for y-axis
    df: boolean
        If True, values come from the df_highPCE dataframe
        If False, feature and target are just the names of the lists conatining the values 
    xyline: boolean
        If True, creates a 1:1 line with intercept 0
    trendline: boolean
        if True, draws a trendline
    '''
    
    if df ==True:
        x = df_highPCE[feature]
        y = df_highPCE[target]
    else:
        x = feature
        y = target
    
    ax.scatter(x, y, c='black')
    ax.set_xlabel(xlabel) 
    ax.set_ylabel(ylabel)
    
    if xlabel == 'Scharber PCE':
        r2 = SchPCEdon_highPCE[0]
        r2_SEM = SchPCEdon_highPCE[1]
        text = '$R^2$ = ' + str(r2) + ' ' + u"\u00B1" + ' ' + str(r2_SEM)
        ax.text(0.45, 0.85, text, fontsize = 'medium', transform=ax.transAxes)
    elif xlabel == 'Alharbi PCE':
        r2 = AlharbiPCEDon_highPCE[0]
        r2_SEM = AlharbiPCEDon_highPCE[1]
        text = '$R^2$ = ' + str(r2) + ' ' + u"\u00B1" + ' ' + str(r2_SEM)
        ax.text(0.05, 0.4, text, fontsize = 'medium', transform=ax.transAxes)
    elif xlabel == 'Imamura PCE':
        r2 = ImamuraPCEdon_highPCE[0]
        r2_SEM = ImamuraPCEdon_highPCE[1]
        text = '$R^2$ = ' + str(r2) + ' ' + u"\u00B1" + ' ' + str(r2_SEM)
        ax.text(0.45, 0.85, text, fontsize = 'medium', transform=ax.transAxes)
    elif xlabel == 'OPEP / B3LYP':
        r2 = OPEP_B3LYP_PCE_pred_highPCE[0]
        r2_SEM = OPEP_B3LYP_PCE_pred_highPCE[1]
        text = '$R^2$ = ' + str(r2) + ' ' + u"\u00B1" + ' ' + str(r2_SEM)
        ax.text(0.05, 0.85, text, fontsize = 'medium', transform=ax.transAxes)
    elif xlabel == 'OPEP / sTD-DFT':
        r2 = OPEP_sTDDFT_PCE_pred_highPCE[0]
        r2_SEM = OPEP_sTDDFT_PCE_pred_highPCE[1]
        text = '$R^2$ = ' + str(r2) + ' ' + u"\u00B1" + ' ' + str(r2_SEM)
        ax.text(0.05, 0.85, text, fontsize = 'medium', transform=ax.transAxes)
    else:
        return "not valid model name"
    
    if trendline == True:
        # calc the trendline
        z = np.polyfit(x, y, 1)
        p = np.poly1d(z)
        ax.plot(x,p(x),"r--")
    
    if xyline == True:
        lims = [
        np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
        np.max([ax.get_xlim(), ax.get_ylim()])  # max of both axes
        ]
        ax.plot(lims, lims, 'c--', alpha=0.75, zorder=0)
        ax.set_aspect('equal')
        ax.set_xlim(lims)
        ax.set_ylim(lims)

## Correlation plots for prediction models vs experimental values

In [ ]:
fig = plt.figure(figsize=(18, 8))

ax1 = plt.subplot2grid(shape=(2,9), loc=(0,0), colspan=3)
ax2 = plt.subplot2grid((2,9), (0,3), colspan=3)
ax3 = plt.subplot2grid((2,9), (0,6), colspan=3)
ax4 = plt.subplot2grid((2,9), (1,1), colspan=3)
ax5 = plt.subplot2grid((2,9), (1,5), colspan=3)

make_scatter_plot(ax1,'ScharberPCEdon', 'ExperimentalPCE', 'Scharber PCE', 'Experimental PCE',df = True, trendline=True)
make_scatter_plot(ax2,'ImamuraPCEdon', 'ExperimentalPCE', 'Imamura PCE', 'Experimental PCE', df  = True, trendline=True)
make_scatter_plot(ax3,'AlharbiPCEDon', 'ExperimentalPCE', 'Alharbi PCE', 'Experimental PCE', df  = True, trendline=True)
make_scatter_plot(ax4,LassoPCE_highPCE[-1], df_highPCE['ExperimentalPCE'], 'OPEP / B3LYP', 'Experimental PCE', trendline=True)
make_scatter_plot(ax5,LassoPCE_sTD_highPCE[-1], df_highPCE['ExperimentalPCE'], 'OPEP / sTD-DFT', 'Experimental PCE', trendline=True)

plt.tight_layout()
plt.subplots_adjust(top = 0.93, wspace = 1, hspace = 0.4)
plt.savefig(os.path.join(tgt_dir, 'correlations_highPCE.svg'))
plt.savefig(os.path.join(tgt_dir, 'correlations_highPCE.png'))
plt.savefig(os.path.join(tgt_dir, 'correlations_highPCE.pdf'))
plt.show()

In [ ]:
# scatter plots showing how the models (trained on high PCE), perform on all ranges of PCE
def CrossValModels(predicted, target):
    # predicted is the list of predicted values. Ex: df['ScharberPCEtotal'] or PerrisPCEcalc
    # target is the target experimental value. Ex: df['ExperimentalPCE']

    X = predicted
    y = target
    score = cross_val_score(SMWrapper(sm.OLS), X, y, cv=5, scoring = 'r2')
    SEM_r2 = round((score.std() / math.sqrt(5)) , 3)
    #r2 = str(round(score.mean(), 3)) + ' +/- ' +  str(SEM_r2)
    r2 = round(float(score.mean()), 3)
    
    score = cross_val_score(SMWrapper(sm.OLS), X, y, cv=5, scoring = 'neg_mean_absolute_error')
    score = np.multiply(score, -1)
    SEM_MAE = round((score.std() / math.sqrt(5)) , 2)
    #MAE = str(round(score.mean(), 2)) + ' +/- ' +  str(SEM_MAE)
    MAE = round(score.mean(), 2) 
    
    score = cross_val_score(SMWrapper(sm.OLS), X, y, cv=5, scoring = 'neg_mean_squared_error')
    score = np.sqrt(score * -1)
    SEM_RMSE = round((score.std() / math.sqrt(5)) , 2)
    #RMSE = str(round(score.mean(), 2)) + ' +/- ' +  str(SEM_RMSE)
    RMSE = round(score.mean(), 2)
    
    MAPE_scorer = make_scorer(mean_absolute_percentage_error)
    score = cross_val_score(SMWrapper(sm.OLS), X, y, cv=5, scoring = MAPE_scorer)
    score = score * 100
    SEM_MAPE = round((score.std() / math.sqrt(5)) , 2)
    MAPE = round(score.mean(), 2)

    return round(r2,2), SEM_r2, MAE, SEM_MAE, RMSE, SEM_RMSE, MAPE, SEM_MAPE

fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize = (15, 5))

sTDDFT_expanded_PCE = []
sTDDFT_PCE = []
B3LYP_PCE = []
experimental_PCE = []

for index, row in df.iterrows():
    sTDDFT_expanded_eq = -33.079899985 + (4.2551151709174313 * row['sTDDFTdeltaHOMO']) + (1.3773119416209663 * row['sTDDFTsummedoscs'])+ (-0.45867638953985362 * row['donsTDDFTsummedoscs'])+ (0.17352602219989174 * row['donsTDDFTabsFOM'])+ ( 2.449065076623004 * row['sTDDFTDonElectrodonating'])+ (0.0009507945724509905 * row['sTDDFTdonfirstengtranswavenumber'])
    sTDDFT_expanded_PCE.append(sTDDFT_expanded_eq)
    
    sTDDFT_eq = 99.8345577608 + (-7.5346772332273959 * row['sTDDFTdeltaLUMO']) + ( 5.4946743170844936 * row['sTDDFTdeltaHOMO']) + (-1.6636022959307213 * row['donsTDDFTdipolemoment']) + (1.5677462537650493 * row['sTDDFTsummedoscs']) + (0.097999426749019958 * row['donsTDDFTabsFOM']) + ( -0.0047307991771359317 * row['sTDDFTdonlowestengtranswavenumber'])
    sTDDFT_PCE.append(sTDDFT_eq)
    
    B3LYP_eq = -136.521585951 + (6.4223592219051397 * row['AccTriplet']) + ( 0.91736200720853223 * row['DeltaDipMom']) + (-0.00046578842135047271 * row['AccEnergyTransitionWavenumber']) + (-0.0036095492932883208 * row['Polarizability']) + (8.3069007344054988 * row['DonElectrodonating']) + ( 0.0038950158798875456 * row['don_lowest_transition_wavenumber'])
    B3LYP_PCE.append(B3LYP_eq)
    
    experimental_PCE.append(row['ExperimentalPCE'])
    
crossed_B3LYP = CrossValModels(B3LYP_PCE, experimental_PCE)
print(crossed_B3LYP)
crossed_sTDDFT = CrossValModels(sTDDFT_PCE, experimental_PCE)
print(crossed_sTDDFT)
crossed_sTDDFTexpanded = CrossValModels(sTDDFT_expanded_PCE, experimental_PCE)
print(crossed_sTDDFTexpanded)
    
ax1.scatter(B3LYP_PCE, experimental_PCE, c='black')
ax1.set_xlabel('OPEP/B3LYP PCE') 
ax1.set_ylabel('Experimental PCE')

# calc the trendline
z = np.polyfit(B3LYP_PCE, experimental_PCE, 1)
p = np.poly1d(z)
ax1.plot(B3LYP_PCE,p(B3LYP_PCE),"r--")

r2 = crossed_B3LYP[0]
r2_SEM = round(crossed_B3LYP[1], 2)
ax1textr2 = '$R^2$ = ' + str(r2) + ' ' + u"\u00B1" + ' ' + str(r2_SEM)
ax1.text(0.05, 0.9, ax1textr2, fontsize = 15, transform=ax1.transAxes)
rmse = round(crossed_B3LYP[4], 2)
rmse_SEM = round(crossed_B3LYP[5], 2)
ax1textrmse = 'RMSE = ' + str(rmse) + ' ' + u"\u00B1" + ' ' + str(rmse_SEM)
ax1.text(0.05, 0.8, ax1textrmse, fontsize = 15, transform=ax1.transAxes)

ax2.scatter(sTDDFT_PCE, experimental_PCE, c='black')
ax2.set_xlabel('OPEP/sTD-DFT PCE') 
ax2.set_ylabel('Experimental PCE')

# calc the trendline
z = np.polyfit(sTDDFT_PCE, experimental_PCE, 1)
p = np.poly1d(z)
ax2.plot(sTDDFT_PCE,p(sTDDFT_PCE),"r--")

r2 = crossed_sTDDFT[0]
r2_SEM = round(crossed_sTDDFT[1], 2)
ax2textr2 = '$R^2$ = ' + str(r2) + ' ' + u"\u00B1" + ' ' + str(r2_SEM)
ax2.text(0.05, 0.9, ax2textr2, fontsize = 15, transform=ax2.transAxes)
rmse = round(crossed_sTDDFT[4], 2)
rmse_SEM = round(crossed_sTDDFT[5], 2)
ax2textrmse = 'RMSE = ' + str(rmse) + ' ' + u"\u00B1" + ' ' + str(rmse_SEM)
ax2.text(0.05, 0.8, ax2textrmse, fontsize = 15, transform=ax2.transAxes)


ax3.scatter(sTDDFT_expanded_PCE, experimental_PCE, c='black')
ax3.set_xlabel('OPEP/sTD-DFT Expanded') 
ax3.set_ylabel('Experimental PCE')

# calc the trendline
z = np.polyfit(sTDDFT_expanded_PCE, experimental_PCE, 1)
p = np.poly1d(z)
ax3.plot(sTDDFT_expanded_PCE,p(sTDDFT_expanded_PCE),"r--")

r2 = crossed_sTDDFTexpanded[0]
r2_SEM = round(crossed_sTDDFTexpanded[1], 2)
ax2textr2 = '$R^2$ = ' + str(r2) + ' ' + u"\u00B1" + ' ' + str(r2_SEM)
ax3.text(0.05, 0.9, ax2textr2, fontsize = 15, transform=ax3.transAxes)
rmse = round(crossed_sTDDFTexpanded[4], 2)
rmse_SEM = round(crossed_sTDDFTexpanded[5], 2)
ax2textrmse = 'RMSE = ' + str(rmse) + ' ' + u"\u00B1" + ' ' + str(rmse_SEM)
ax3.text(0.05, 0.8, ax2textrmse, fontsize = 15, transform=ax3.transAxes)


plt.tight_layout()
plt.savefig(os.path.join(tgt_dir, 'correlation_highPCEmodels_vs_allPCE.png'))
plt.savefig(os.path.join(tgt_dir, 'correlation_highPCEmodels_vs_allPCE.pdf'), dpi=300)

plt.show()

## T-Test comparing the various models

Create arrays of PCE of each model trained on high PCE on high PCE dataset

In [47]:
sTDDFT_expanded_PCE = []
sTDDFT_PCE = []
B3LYP_PCE = []

for index, row in df_highPCE.iterrows():
    sTDDFT_expanded_eq = -33.079899985 + (4.2551151709174313 * row['sTDDFTdeltaHOMO']) + (1.3773119416209663 * row['sTDDFTsummedoscs'])+ (-0.45867638953985362 * row['donsTDDFTsummedoscs'])+ (0.17352602219989174 * row['donsTDDFTabsFOM'])+ ( 2.449065076623004 * row['sTDDFTDonElectrodonating'])+ (0.0009507945724509905 * row['sTDDFTdonfirstengtranswavenumber'])
    sTDDFT_expanded_PCE.append(sTDDFT_expanded_eq)
    
    sTDDFT_eq = 99.8345577608 + (-7.5346772332273959 * row['sTDDFTdeltaLUMO']) + ( 5.4946743170844936 * row['sTDDFTdeltaHOMO']) + (-1.6636022959307213 * row['donsTDDFTdipolemoment']) + (1.5677462537650493 * row['sTDDFTsummedoscs']) + (0.097999426749019958 * row['donsTDDFTabsFOM']) + ( -0.0047307991771359317 * row['sTDDFTdonlowestengtranswavenumber'])
    sTDDFT_PCE.append(sTDDFT_eq)
    
    B3LYP_eq = -136.521585951 + (6.4223592219051397 * row['AccTriplet']) + ( 0.91736200720853223 * row['DeltaDipMom']) + (-0.00046578842135047271 * row['AccEnergyTransitionWavenumber']) + (-0.0036095492932883208 * row['Polarizability']) + (8.3069007344054988 * row['DonElectrodonating']) + ( 0.0038950158798875456 * row['don_lowest_transition_wavenumber'])
    B3LYP_PCE.append(B3LYP_eq)

In [48]:
stats.ttest_ind(B3LYP_PCE, sTDDFT_PCE)

Ttest_indResult(statistic=-0.02287360103145494, pvalue=0.9817897332023132)

In [49]:
stats.ttest_ind(sTDDFT_PCE,sTDDFT_expanded_PCE)

Ttest_indResult(statistic=0.15063209396242494, pvalue=0.8805231364877648)

## Plots to compare models

In [ ]:
def AbsoluteError(predicted, experimental, percent_error = False):
    # predicted is the predicted values. Ex: df['ScharberFF']
    # experimental is the target experimental values. Ex: df['ExperimentalFF']
    # returns are list of errors
    
    absolute_errors = []
    for x in range(len(experimental)):
        if percent_error == True:
            absolute_errors.append(((predicted[x] - experimental[x])/experimental[x])*100)
        else:
            absolute_errors.append(predicted[x] - experimental[x])
    return absolute_errors

def make_violinplot(ax, error_list, label_list, title, matchcolor=False):
    # ax is which axes on the figure you want. Ex: ax1
    # error_list is the list of deviations Ex. FF_err_list
    # label_list is the list of labels for x-axis. Make sure it is same order as error_list
    # y_min and y_max are the limits for the y-axis
    # title is a string for subplot titles. Ex: 'FF'
    #
    # returns a plt object

#    ax.grid(b = True, axis = 'y', ls = '--')
    ax.axhline(y = 0, color = 'dimgrey', linestyle = ':')
    
    if matchcolor==True:
        if title == 'Jsc':
            Jsc_palette = [sns.husl_palette(10)[0], sns.husl_palette(10)[2], sns.husl_palette(10)[6], sns.husl_palette(10)[8]]
            sns.violinplot(data = error_list, showextrema = True, showmedians = True, palette = Jsc_palette, ax = ax)
        else:
            sns.violinplot(data = error_list, showextrema = True, showmedians = True, palette = 'husl', ax = ax)
    else:
        sns.violinplot(data = error_list, showextrema = True, showmedians = True, palette = 'husl', ax = ax)
    ax.set_xticklabels(label_list, rotation = 45, ha = 'right')
    ax.set_title(title, weight = 'bold')
    if title == 'FF':
        ax.set_ylabel('Deviation (%)')
    elif title == 'Voc':
        ax.set_ylabel('Deviation (V)')
    elif title == 'Jsc':
        ax.set_ylabel('Deviation ($\mathregular{mA/cm^{2}}$)')
    elif title == 'PCE':
        ax.set_ylabel('Deviation (%)')
    
    return plt


### Violin plot of FF, Voc, Jsc, and PCE with deviations (predicted - experimental) for all PCEs

In [ ]:
ScharberFFerr = AbsoluteError(df['ScharberFF'], df['ExperimentalFF'])
AlharbiFFDonerr = AbsoluteError(df['AlharbiFFDon'], df['ExperimentalFF'])
AlharbiFFAccerr = AbsoluteError(df['AlharbiFFAcc'], df['ExperimentalFF'])
ImamuraFFerr = AbsoluteError(df['ImamuraFF'], df['ExperimentalFF'])
OPEP_B3LYP_FF_err = AbsoluteError(LassoFF[-1], df['ExperimentalFF'])
OPEP_sTDDFT_FF_err = AbsoluteError(LassoFF_sTD[-1], df['ExperimentalFF'])
OPEP_sTDDFT_Expanded_FF_err = AbsoluteError(LassoFF_sTD_all[-1], df_sTD['ExperimentalFF'])

FF_err_list = [ScharberFFerr, ImamuraFFerr, AlharbiFFDonerr, AlharbiFFAccerr,  OPEP_B3LYP_FF_err, OPEP_sTDDFT_FF_err, OPEP_sTDDFT_Expanded_FF_err]
FF_list = ['Scharber','Imamura', 'Alharbi (Don)', 'Alharbi (Acc)', 'OPEP / B3LYP', 'OPEP / sTD-DFT', 'OPEP / sTD-DFT Expanded']


AlharbiVocAccErr = AbsoluteError(df['AlharbiVocAcc'], df['ExperimentalVoc'])
AlharbiVocDonErr = AbsoluteError(df['AlharbiVocDon'], df['ExperimentalVoc'])
ImamuraVocErr = AbsoluteError(df['ImamuraVoc'], df['ExperimentalVoc'])
ScharberVocErr = AbsoluteError(df['ScharberVoc'], df['ExperimentalVoc'])
OPEP_B3LYP_Voc_err = AbsoluteError(LassoVoc[-1], df['ExperimentalVoc'])
OPEP_sTDDFT_Voc_err = AbsoluteError(LassoVoc_sTD[-1], df['ExperimentalVoc'])
OPEP_sTDDFT_Expanded_Voc_err = AbsoluteError(LassoVoc_sTD_all[-1], df_sTD['ExperimentalVoc'])

Voc_err_list = [ScharberVocErr,ImamuraVocErr, AlharbiVocDonErr, AlharbiVocAccErr, OPEP_B3LYP_Voc_err, OPEP_sTDDFT_Voc_err, OPEP_sTDDFT_Expanded_Voc_err]
Voc_list = ['Scharber','Imamura','Alharbi (Don)', 'Alharbi (Acc)', 'OPEP / B3LYP', 'OPEP / sTD-DFT', 'OPEP / sTD-DFT Expanded']


ScharberJscAccErr = AbsoluteError(df['ScharberJscAcc'], df['ExperimentalJsc'])
ImamuraTotalJscErr = AbsoluteError(df['ImamuraTotalJsc'], df['ExperimentalJsc'])
ImamuraJscAccErr = AbsoluteError(df['ImamuraJscAcc'], df['ExperimentalJsc'])
ImamuraJscDonErr = AbsoluteError(df['ImamuraJscDon'], df['ExperimentalJsc'])
ScharberTotalJscErr = AbsoluteError(df['ScharberTotalJsc'], df['ExperimentalJsc'])
ScharberJscDonErr = AbsoluteError(df['ScharberJscDon'], df['ExperimentalJsc'])
ScharberJsclowestoptbgErr = AbsoluteError(df['ScharberJsclowestoptbg'], df['ExperimentalJsc'])
OPEP_B3LYP_Jsc_err = AbsoluteError(LassoJsc[-1], df['ExperimentalJsc'])
OPEP_sTDDFT_Jsc_err = AbsoluteError(LassoJsc_sTD[-1], df['ExperimentalJsc'])
OPEP_sTDDFT_Expanded_Jsc_err = AbsoluteError(LassoJsc_sTD_all[-1], df_sTD['ExperimentalJsc'])

Jsc_err_list = [ScharberJscDonErr,ScharberJscAccErr, ScharberTotalJscErr, ScharberJsclowestoptbgErr, ImamuraJscDonErr,ImamuraJscAccErr, ImamuraTotalJscErr, OPEP_B3LYP_Jsc_err, OPEP_sTDDFT_Jsc_err, OPEP_sTDDFT_Expanded_Jsc_err]
Jsc_list = ['Scharber (Don)', 'Scharber (Acc)', 'Scharber (Total)', 'Scharber (Low Eg)', 'Imamura (Don)','Imamura (Acc)','Imamura (Total)', 'OPEP / B3LYP', 'OPEP / sTD-DFT', 'OPEP / sTD-DFT Expanded']


SchPCEdonErr = AbsoluteError(df['ScharberPCEdon'], df['ExperimentalPCE'])
SchPCEaccErr = AbsoluteError(df['ScharberPCEacc'], df['ExperimentalPCE'])
SchPCEtotalErr = AbsoluteError(df['ScharberPCEtotal'], df['ExperimentalPCE'])
SchPCElowbgErr = AbsoluteError(df['ScharberPCElowestoptbg'], df['ExperimentalPCE'])
ImamuraPCEdonErr = AbsoluteError(df['ImamuraPCEdon'], df['ExperimentalPCE'])
ImamuraPCEaccErr = AbsoluteError(df['ImamuraPCEacc'], df['ExperimentalPCE'])
ImamuraPCEtotErr = AbsoluteError(df['ImamuraPCEtotal'], df['ExperimentalPCE'])
AlharbiPCEDonErr = AbsoluteError(df['AlharbiPCEDon'], df['ExperimentalPCE'])
AlharbiPCEAccErr = AbsoluteError(df['AlharbiPCEAcc'], df['ExperimentalPCE'])
OPEP_B3LYP_PCE_pred_err = AbsoluteError(LassoPCE[-1], df['ExperimentalPCE'])
OPEP_sTDDFT_PCE_pred_err = AbsoluteError(LassoPCE_sTD[-1], df['ExperimentalPCE'])
OPEP_sTDDFT_Expanded_PCE_pred_err = AbsoluteError(LassoPCE_sTD_all[-1], df_sTD['ExperimentalPCE'])

OPEP_B3LYP_PCE_calc_err = AbsoluteError(OPEP_B3LYP_PCE_Calc_values, df['ExperimentalPCE'])
OPEP_B3LYP_PCE_FF70_err = AbsoluteError(OPEP_B3LYP_PCE_FF70_values, df['ExperimentalPCE'])
OPEP_sTDDFT_PCE_calc_err = AbsoluteError(OPEP_sTDDFT_PCE_calc_values, df['ExperimentalPCE'])
OPEP_sTDDFT_PCE_FF70_err = AbsoluteError(OPEP_sTDDFT_PCE_FF70_values, df['ExperimentalPCE'])
OPEP_sTDDFT_Expanded_PCE_calc_err = AbsoluteError(OPEP_sTDDFT_PCE_calc_expanded_values, df_sTD['ExperimentalPCE'])
OPEP_sTDDFT_Expanded_PCE_FF70_err = AbsoluteError(OPEP_sTDDFT_PCE_FF70_expanded_values, df_sTD['ExperimentalPCE'])

PCE_err_list = [
    SchPCEdonErr, SchPCEaccErr, SchPCEtotalErr, SchPCElowbgErr, ImamuraPCEdonErr, ImamuraPCEaccErr, ImamuraPCEtotErr, AlharbiPCEDonErr, AlharbiPCEAccErr,
    OPEP_B3LYP_PCE_pred_err, OPEP_sTDDFT_PCE_pred_err, OPEP_sTDDFT_Expanded_PCE_pred_err,
    OPEP_B3LYP_PCE_calc_err, OPEP_sTDDFT_PCE_calc_err, OPEP_sTDDFT_Expanded_PCE_calc_err
]
PCE_list = [
    'Scharber (Don)', 'Scharber (Acc)', 'Scharber (total)', 'Scharber (lowest Eg)', 'Imamura (Don)', 'Imamura (Acc)', 'Imamura (total)', 'modified Alharbi (Don)', 'modified Alharbi (Acc)',
    'OPEP / B3LYP', 'OPEP / sTD-DFT', 'OPEP / sTD-DFT Expanded', 
    'OPEP / B3LYP (calc.)', 'OPEP / sTD-DFT (calc.)', 'OPEP / sTD-DFT Expanded (calc.)'
]

### Violin plots of FF, Voc, Jsc, and PCE with deviations for any experimental PCE

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2, figsize = (22, 14))

make_violinplot(ax1, FF_err_list, FF_list, 'FF' )
make_violinplot(ax2, Voc_err_list, Voc_list, 'Voc')
make_violinplot(ax3, Jsc_err_list, Jsc_list, 'Jsc')
make_violinplot(ax4, PCE_err_list, PCE_list,'PCE')

#plt.subplots_adjust(top = 0.95, bottom = 0.05, wspace = 0.2, hspace = 0.85)
plt.tight_layout()
plt.savefig(os.path.join(tgt_dir, 'violinplots_all_models.pdf'), dpi=300)
plt.savefig(os.path.join(tgt_dir, 'violinplots_all_models.png'), dpi=300)
plt.show()

### Violin plots of FF, Voc, Jsc, and PCE with deviations for experimetnal PCEs > 9%

In [ ]:
ScharberFFerr = AbsoluteError(df_highPCE['ScharberFF'], df_highPCE['ExperimentalFF'])
AlharbiFFDonerr = AbsoluteError(df_highPCE['AlharbiFFDon'], df_highPCE['ExperimentalFF'])
ImamuraFFerr = AbsoluteError(df_highPCE['ImamuraFF'], df_highPCE['ExperimentalFF'])
OPEP_B3LYP_FF_err = AbsoluteError(LassoFF_highPCE[-1], df_highPCE['ExperimentalFF'])
OPEP_sTDDFT_FF_err = AbsoluteError(LassoFF_sTD_highPCE[-1], df_highPCE['ExperimentalFF'])

FF_err_list_highPCE = [ScharberFFerr,ImamuraFFerr, AlharbiFFDonerr, OPEP_B3LYP_FF_err, OPEP_sTDDFT_FF_err]
FF_list = ['Scharber', 'Imamura', 'Alharbi','OPEP / B3LYP', 'OPEP / sTD-DFT']


AlharbiVocDonErr = AbsoluteError(df_highPCE['AlharbiVocDon'], df_highPCE['ExperimentalVoc'])
ImamuraVocErr = AbsoluteError(df_highPCE['ImamuraVoc'], df_highPCE['ExperimentalVoc'])
ScharberVocErr = AbsoluteError(df_highPCE['ScharberVoc'], df_highPCE['ExperimentalVoc'])
OPEP_B3LYP_Voc_err = AbsoluteError(LassoVoc_highPCE[-1], df_highPCE['ExperimentalVoc'])
OPEP_sTDDFT_Voc_err = AbsoluteError(LassoVoc_sTD_highPCE[-1], df_highPCE['ExperimentalVoc'])

Voc_err_list_highPCE = [ScharberVocErr, ImamuraVocErr, AlharbiVocDonErr, OPEP_B3LYP_Voc_err, OPEP_sTDDFT_Voc_err]
Voc_list = ['Scharber', 'Imamura', 'Alharbi', 'OPEP / B3LYP', 'OPEP / sTD-DFT']


ImamuraJscDonErr = AbsoluteError(df_highPCE['ImamuraJscDon'], df_highPCE['ExperimentalJsc'])
ScharberJscDonErr = AbsoluteError(df_highPCE['ScharberJscDon'], df_highPCE['ExperimentalJsc'])
OPEP_B3LYP_Jsc_err = AbsoluteError(LassoJsc_highPCE[-1], df_highPCE['ExperimentalJsc'])
OPEP_sTDDFT_Jsc_err = AbsoluteError(LassoJsc_sTD_highPCE[-1], df_highPCE['ExperimentalJsc'])

Jsc_err_list_highPCE = [ScharberJscDonErr, ImamuraJscDonErr, OPEP_B3LYP_Jsc_err, OPEP_sTDDFT_Jsc_err]
Jsc_list = ['Scharber', 'Imamura', 'OPEP / B3LYP', 'OPEP / sTD-DFT']


SchPCEdonErr = AbsoluteError(df_highPCE['ScharberPCEdon'], df_highPCE['ExperimentalPCE']) 
ImamuraPCEdonErr = AbsoluteError(df_highPCE['ImamuraPCEdon'], df_highPCE['ExperimentalPCE'])
AlharbiPCEDonErr = AbsoluteError(df_highPCE['AlharbiPCEDon'], df_highPCE['ExperimentalPCE'])
OPEP_B3LYP_PCE_err = AbsoluteError(LassoPCE_highPCE[-1], df_highPCE['ExperimentalPCE'])
OPEP_sTDDFT_PCE_err = AbsoluteError(LassoPCE_sTD_highPCE[-1], df_highPCE['ExperimentalPCE'])
OPEP_sTDDFT_Expanded_PCE_err = AbsoluteError(LassoPCE_sTD_highPCE_all[-1], df_sTD_highPCE['ExperimentalPCE'])

PCE_err_list_highPCE = [SchPCEdonErr, ImamuraPCEdonErr,AlharbiPCEDonErr, OPEP_B3LYP_PCE_err, OPEP_sTDDFT_PCE_err]
PCE_list = ['Scharber', 'Imamura', 'Modified Alharbi', 'OPEP / B3LYP' , 'OPEP / sTD-DFT']

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2, figsize = (12, 12))

make_violinplot(ax1,FF_err_list_highPCE, FF_list, 'FF' )
make_violinplot(ax2, Voc_err_list_highPCE, Voc_list, 'Voc')
make_violinplot(ax3, Jsc_err_list_highPCE, Jsc_list, 'Jsc', matchcolor = True)
make_violinplot(ax4, PCE_err_list_highPCE, PCE_list,'PCE')

plt.tight_layout()
plt.subplots_adjust(top = 0.93, wspace = 0.4, hspace = 0.7)

plt.savefig(os.path.join(tgt_dir, 'violinplots_highPCE.png'), dpi=300)
plt.savefig(os.path.join(tgt_dir, 'violinplots_highPCE.pdf'), dpi=300)
plt.show()

## Correlation plots between DFT and sTD-DFT

In [ ]:
def scatterplot_bigfig(feature, target, df, xlabel, ylabel, xyline=False, trendline=False, ax=False):
    x = df[feature]
    y = df[target]
    
    if ax == False:
        fig, ax = plt.subplots()
    ax.scatter(x, y, c='black') 
    ax.set_xlabel(xlabel) 
    ax.set_ylabel(ylabel)
    
    if trendline == True:
        # calc the trendline
        z = np.polyfit(x, y, 1)
        p = np.poly1d(z)
        pylab.plot(x,p(x),"r--")

    if xyline == True:
        lims = [
        np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
        np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
        ]

        ax.plot(lims, lims, 'c--', alpha=0.75, zorder=0)
        ax.set_aspect('equal')
        ax.set_xlim(lims)
        ax.set_ylim(lims)
    
    return plt

In [ ]:
fig, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(2, 3, figsize = (18, 12))

scatterplot_bigfig('AccHOMO', 'sTDDFTHOMO', df, 'B3LYP HOMO (eV)', 'sTD-CAM-B3LYP HOMO (eV)', xyline=True, ax = ax1)
scatterplot_bigfig('acc_lowest_transition_wavenumber', 'sTDDFTacclowestengtranswavenumber', df,'TD-B3LYP Excitation Energy ($\mathregular{cm^{-1}}$)', 'sTD-CAM-B3LYP Excitation Energy ($\mathregular{cm^{-1}}$)', xyline=True, ax = ax2)
scatterplot_bigfig('AccDeltaHOMO', 'sTDDFTdeltaHOMO', df,'B3LYP \u0394HOMO (eV)', 'sTD-CAM-B3LYP \u0394HOMO (eV)', xyline=True, ax=ax3)
scatterplot_bigfig('AccDeltaLUMO', 'sTDDFTdeltaLUMO', df,'B3LYP \u0394LUMO (eV)', 'sTD-CAM-B3LYP \u0394LUMO (eV)', xyline=True, ax=ax4)
scatterplot_bigfig('GSDipMom', 'sTDDFTdipolemoment', df,'B3LYP Dipole Moment (Debye)', 'sTD-CAM-B3LYP Dipole Moment (Debye)', xyline=True, ax=ax5)
scatterplot_bigfig('acc_highest_oscs_under_ten', 'sTDDFTacchighestoscsunderten', df,'TD-B3LYP Oscillator Strength', 'sTD-CAM-B3LYP Oscillator Strength', xyline=True, ax=ax6)

plt.subplots_adjust(top = 0.93, wspace = 0.3, hspace = 0.4)
plt.tight_layout()
plt.savefig(os.path.join(tgt_dir, 'DFT_vs_sTDDFT.pdf'))
plt.savefig(os.path.join(tgt_dir, 'DFT_vs_sTDDFT.png'), dpi=300)
plt.show()